In [ ]:
import os
import sqlite3
import datetime
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import customtkinter as ctk
from PIL import Image, ImageTk
import io
import subprocess
import sys
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
import json
from dateutil import parser

def safe_parse(raw):
    if raw:
        try:
            return parser.parse(raw)
        except (ValueError, TypeError):
            return None
    return None

# Set appearance mode and default color theme
ctk.set_appearance_mode("Dark")  # Modes: "System" (standard), "Dark", "Light"
ctk.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"

# Create database directory if it doesn't exist
if not os.path.exists("data"):
    os.makedirs("data")

# Set up the database
DB_PATH = "data/eira_notes.db"

def init_db():
    # Connect to the database (will create it if it doesn't exist)
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # Create User table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        email TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        role TEXT NOT NULL DEFAULT 'physiotherapist', 
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # Add role column to existing databases if missing
    cursor.execute("PRAGMA table_info(users)")
    existing_columns = [row[1] for row in cursor.fetchall()]
    if 'role' not in existing_columns:
        cursor.execute("ALTER TABLE users ADD COLUMN role TEXT NOT NULL DEFAULT 'physiotherapist'")
        cursor.execute("UPDATE users SET role = 'physiotherapist' WHERE role IS NULL")

    
    # Create Patient table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS patients (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        date_of_birth DATE NOT NULL,
        gender TEXT,
        id_number TEXT NOT NULL,
        phone TEXT,
        phone2 TEXT,
        email TEXT,
        address TEXT,
        po_box TEXT,
        medical_aid_name TEXT NOT NULL,
        medical_aid_number TEXT,
        medical_aid_id TEXT,
        medical_history TEXT,
        diagnosis TEXT NOT NULL,
        icd_code TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        user_id INTEGER NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users (id)
    )
    ''')

    try:
        cursor.execute("ALTER TABLE patients ADD COLUMN icd_code TEXT")
    except sqlite3.OperationalError:
        pass

    # Create SOAP Note table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS soap_notes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL,
        subjective TEXT NOT NULL,
        objective TEXT NOT NULL,
        action TEXT NOT NULL,
        plan TEXT NOT NULL,
        treatment_provided TEXT,
        patient_response TEXT,
        goals_progress TEXT,
        patient_id INTEGER NOT NULL,
        created_by INTEGER NOT NULL,
        FOREIGN KEY (patient_id) REFERENCES patients (id),
        FOREIGN KEY (created_by) REFERENCES users (id)
    )
    ''')
    
    # Create Appointment table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS appointments (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE NOT NULL,
        time TIME NOT NULL,
        duration INTEGER DEFAULT 60,
        notes TEXT,
        status TEXT DEFAULT 'scheduled',
        patient_id INTEGER NOT NULL,
        user_id INTEGER NOT NULL,
        FOREIGN KEY (patient_id) REFERENCES patients (id),
        FOREIGN KEY (user_id) REFERENCES users (id)
    )
    ''')

    # Ensure required columns exists in patients table
    cursor.execute("PRAGMA table_info(patients)")
    patient_columns = [col[1] for col in cursor.fetchall()]
    if 'id_number' not in patient_columns:
        cursor.execute("ALTER TABLE patients ADD COLUMN id_number TEXT")
    if 'phone2' not in patient_columns:
        cursor.execute("ALTER TABLE patients ADD COLUMN phone2 TEXT")
    if 'po_box' not in patient_columns:
        cursor.execute("ALTER TABLE patients ADD COLUMN po_box TEXT")
    if 'medical_aid_id' not in patient_columns:
        cursor.execute("ALTER TABLE patients ADD COLUMN medical_aid_id TEXT")

    
    # Create a default admin user if it doesn't exist
    cursor.execute("SELECT id FROM users WHERE username = 'admin'")
    if not cursor.fetchone():
        from werkzeug.security import generate_password_hash
        password_hash = generate_password_hash("admin123")
        cursor.execute(
            "INSERT INTO users (username, email, password_hash, first_name, last_name) VALUES (?, ?, ?, ?, ?)",
            (
                "admin",
                "admin@example.com",
                password_hash,
                "Admin",
                "User",
                "administrator",
            ),
        )
    
    conn.commit()
    conn.close()

# Initialize the database
init_db()

# Authentication functions
def authenticate(username, password):
    from werkzeug.security import check_password_hash
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    cursor.execute("SELECT id, password_hash, first_name, last_name, role FROM users WHERE username = ?", (username,))
    user = cursor.fetchone()
    conn.close()
    
    if user and check_password_hash(user[1], password):
        return {
            "id": user[0],
            "first_name": user[2],
            "last_name": user[3],
            "role": user[4],
        }
    return None

def register_user(username: str,
    email: str,
    password: str,
    first_name: str,
    last_name: str,
    role: str = "physiotherapist",
    ):
    from werkzeug.security import generate_password_hash
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        password_hash = generate_password_hash(password)
        cursor.execute(
            "INSERT INTO users (username, email, password_hash, first_name, last_name, role) VALUES (?, ?, ?, ?, ?, ?)",
            (username, email, password_hash, first_name, last_name, role)
        )
        conn.commit()
        result = {"success": True, "message": "User registered successfully"}
    except sqlite3.IntegrityError as e:
        conn.rollback()
        if "username" in str(e):
            result = {"success": False, "message": "Username already exists"}
        elif "email" in str(e):
            result = {"success": False, "message": "Email already exists"}
        else:
            result = {"success": False, "message": f"Registration error: {str(e)}"}
    finally:
        conn.close()
    
    return result


def get_all_users():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    cursor.execute(
        "SELECT id, username, email, first_name, last_name, role FROM users ORDER BY username"
    )
    users = [dict(row) for row in cursor.fetchall()]
    conn.close()
    return users


def update_user_role(user_id, role):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    try:
        cursor.execute("UPDATE users SET role = ? WHERE id = ?", (role, user_id))
        conn.commit()
        return {"success": True, "message": "Role updated"}
    except Exception as e:
        conn.rollback()
        return {"success": False, "message": f"Error updating role: {e}"}
    finally:
        conn.close()

def get_patients(user_id):
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    cursor.execute(
        "SELECT * FROM patients WHERE user_id = ? ORDER BY last_name, first_name",
        (user_id,),
    )
    patients = [dict(row) for row in cursor.fetchall()]
    conn.close()
    return patients


def get_patient(patient_id):
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
    row = cursor.fetchone()
    patient = dict(row) if row else None
    conn.close()
    return patient

def add_patient(data, user_id):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute(
            """INSERT INTO patients (
                first_name, last_name, date_of_birth, id_number, gender, phone, phone2, email,
                address, po_box, medical_aid_name, medical_aid_number, medical_aid_id,
                medical_history, diagnosis, user_id
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            (
                data["first_name"], data["last_name"], data["id_number"], data["date_of_birth"], data["gender"],
                data["phone"], data["phone2"], data["email"], data["address"], data["po_box"],
                data["medical_aid_name"], data["medical_aid_number"], data["medical_aid_id"],
                data["medical_history"], data["diagnosis"], user_id
            )
        )
        conn.commit()
        patient_id = cursor.lastrowid
        result = {"success": True, "message": "Patient added successfully", "patient_id": patient_id}
    except Exception as e:
        conn.rollback()
        result = {"success": False, "message": f"Error adding patient: {str(e)}"}
    finally:
        conn.close()
    
    return result

def update_patient(patient_id, data):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute(
            """UPDATE patients SET
                first_name = ?, last_name = ?, id_number = ?, date_of_birth = ?, gender = ?, phone2 = ?, email = ?,
                address = ?, po_box = ?, medical_aid_name = ?, medical_aid_number = ?, medical_aid_id = ?, medical_history = ?, diagnosis = ?
            WHERE id = ?""",
            (
                data["first_name"], data["last_name"], data["id_number"], data["date_of_birth"], data["gender"],
                data["phone"], data["phone2"], data["email"], data["address"], data["po_box"],
                data["medical_aid_name"], data["medical_aid_number"], data("medical_aid_id"),
                data["medical_history"], data["diagnosis"], patient_id
            )
        )
        conn.commit()
        result = {"success": True, "message": "Patient updated successfully"}
    except Exception as e:
        conn.rollback()
        result = {"success": False, "message": f"Error updating patient: {str(e)}"}
    finally:
        conn.close()
    
    return result

def delete_patient(patient_id):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        # Delete associated records first (due to foreign key constraints)
        cursor.execute("DELETE FROM soap_notes WHERE patient_id = ?", (patient_id,))
        cursor.execute("DELETE FROM appointments WHERE patient_id = ?", (patient_id,))
        cursor.execute("DELETE FROM patients WHERE id = ?", (patient_id,))
        conn.commit()
        result = {"success": True, "message": "Patient and all associated records deleted"}
    except Exception as e:
        conn.rollback()
        result = {"success": False, "message": f"Error deleting patient: {str(e)}"}
    finally:
        conn.close()
    
    return result

# SOAP Note functions
def get_soap_notes(patient_id):
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    cursor.execute(
        """SELECT n.*, u.first_name || '%Y-%m-%d' || u.last_name as therapist_name 
        FROM soap_notes n
        JOIN users u ON n.created_by = u.id
        WHERE n.patient_id = ?
        ORDER BY n.date DESC""",
        (patient_id,)
    )
    row = cursor.fetchone()
    note = [dict(row) for row in cursor.fetchall()]
    conn.close()
    
    return notes

def get_soap_note(note_id):
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    cursor.execute(
        """SELECT n.*, u.first_name || '%Y-%m-%d' || u.last_name as therapist_name 
        FROM soap_notes n
        JOIN users u ON n.created_by = u.id
        WHERE n.id = ?""",
        (note_id,)
    )
    row = cursor.fetchone()
    note = [dict(row) for row in cursor.fetchall()]
    conn.close()
    
    return note

def add_soap_note(data, patient_id, user_id):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute(
            """INSERT INTO soap_notes (
                date, subjective, objective, action, plan, 
                treatment_provided, patient_response, goals_progress,
                patient_id, created_by
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            (
                data["date"], data["subjective"], data["objective"], data["action"], data["plan"],
                data.get("treatment_provided"), data.get("patient_response"), data.get("goals_progress"),
                patient_id, user_id
            )
        )
        conn.commit()
        note_id = cursor.lastrowid
        result = {"success": True, "message": "SOAP note added successfully", "note_id": note_id}
    except Exception as e:
        conn.rollback()
        result = {"success": False, "message": f"Error adding SOAP note: {str(e)}"}
    finally:
        conn.close()
    
    return result

def update_soap_note(note_id, data, user_id):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute(
            """UPDATE soap_notes SET
                date = ?, subjective = ?, objective = ?, action = ?, plan = ?,
                treatment_provided = ?, patient_response = ?, goals_progress = ?
            WHERE id = ? AND created_by""",
            (
                data["date"], data["subjective"], data["objective"], data["action"], data["plan"],
                data.get("treatment_provided"), data.get("patient_response"), data.get("goals_progress"),
                note_id, user_id
            )
        )
        conn.commit()
        result = {"success": True, "message": "SOAP note updated successfully"}
    except Exception as e:
        conn.rollback()
        result = {"success": False, "message": f"Error updating SOAP note: {str(e)}"}
    finally:
        conn.close()
    
    return result

def delete_soap_note(note_id, user_id):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute(
            "DELETE FROM soap_notes WHERE id = ? AND created_by = ?",
            (note_id, user_id),
        )
        conn.commit()
        result = {"success": True, "message": "SOAP note deleted successfully"}
    except Exception as e:
        conn.rollback()
        result = {"success": False, "message": f"Error deleting SOAP note: {str(e)}"}
    finally:
        conn.close()
    
    return result

# Appointment functions
def get_appointments(user_id, filter_type=None):
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    query = """
        SELECT a.*, p.first_name || ' ' || p.last_name as patient_name
        FROM appointments a
        JOIN patients p ON a.patient_id = p.id
        WHERE a.user_id = ?
    """
    
    params = [user_id]
    
    if filter_type == "today":
        query += " AND date(a.date) = date('now')"
    elif filter_type == "upcoming":
        query += " AND (date(a.date) >= date('now') AND a.status = 'scheduled')"
    
    query += " ORDER BY a.date, a.time"
    
    cursor.execute(query, params)
    row = cursor.fetchone()
    appointment = [dict(row) for row in cursor.fetchall()]
    conn.close()
    
    return appointment

def get_appointment(appointment_id):
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    cursor.execute(
        """SELECT a.*, p.first_name || ' ' || p.last_name as patient_name
        FROM appointments a
        JOIN patients p ON a.patient_id = p.id
        WHERE a.id = ?""",
        (appointment_id,)
    )
    row = cursor.fetchone()
    appointment = [dict(row) for row in cursor.fetchall()]
    conn.close()
    
    return appointment

def add_appointment(data, patient_id, user_id):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute(
            """INSERT INTO appointments (
                date, time, duration, notes, status, patient_id, user_id
            ) VALUES (?, ?, ?, ?, ?, ?, ?)""",
            (data["date"], data["time"], data["duration"], data.get("notes"), data["status"], patient_id, user_id)
        )
        conn.commit()
        appointment_id = cursor.lastrowid
        result = {"success": True, "message": "Appointment added successfully", "appointment_id": appointment_id}
    except Exception as e:
        conn.rollback()
        result = {"success": False, "message": f"Error adding appointment: {str(e)}"}
    finally:
        conn.close()
    
    return result

def update_appointment(appointment_id, data):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute(
            """UPDATE appointments SET
                date = ?, time = ?, duration = ?, notes = ?, status = ?
            WHERE id = ?""",
            (data["date"], data["time"], data["duration"], data.get("notes"), data["status"], appointment_id)
        )
        conn.commit()
        result = {"success": True, "message": "Appointment updated successfully"}
    except Exception as e:
        conn.rollback()
        result = {"success": False, "message": f"Error updating appointment: {str(e)}"}
    finally:
        conn.close()
    
    return result

def delete_appointment(appointment_id):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute("DELETE FROM appointments WHERE id = ?", (appointment_id,))
        conn.commit()
        result = {"success": True, "message": "Appointment deleted successfully"}
    except Exception as e:
        conn.rollback()
        result = {"success": False, "message": f"Error deleting appointment: {str(e)}"}
    finally:
        conn.close()
    
    return result

# Report generation function
def generate_word_report(patient_id, user_id):
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    # Get patient info
    cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
    row = cursor.fetchone()
    patient = [dict(row) for row in cursor.fetchall()]
    
    if not patient:
        conn.close()
        return {"success": False, "message": "Patient not found"}
    
    # Get therapist info
    cursor.execute("SELECT * FROM users WHERE id = ?", (user_id,))
    row = cursor.fetchone()
    therapist = dict(row) if row else None
    
    # Get SOAP notes
    cursor.execute(
        """SELECT * FROM soap_notes 
        WHERE patient_id = ? 
        ORDER BY date DESC""",
        (patient_id,)
    )
    soap_notes = [dict(row) for row in cursor.fetchall()]
    
    conn.close()
    
    if not patient or not therapist:
        return {"success": False, "message": "Could not find patient or therapist information"}
    
    # Create a new Document
    doc = Document()
    
    # Set margins for the document
    sections = doc.sections
    for section in sections:
        section.top_margin = Inches(0.75)
        section.bottom_margin = Inches(0.75)
        section.left_margin = Inches(0.75)
        section.right_margin = Inches(0.75)
    
    # Add title
    title = doc.add_paragraph()
    title_run = title.add_run('Physical Therapy Progress Report')
    title_run.bold = True
    title_run.font.size = Pt(16)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Add date
    date_paragraph = doc.add_paragraph()
    date_run = date_paragraph.add_run(f"Date Generated: {datetime.datetime.now().strftime('%B %d, %Y')}")
    date_run.font.size = Pt(10)
    
    doc.add_paragraph()
    
    # Therapist Information
    doc.add_heading('Therapist Information', level=2)
    therapist_table = doc.add_table(rows=2, cols=2)
    therapist_table.style = 'Table Grid'
    
    # Add therapist details
    therapist_table.cell(0, 0).text = "Name:"
    therapist_table.cell(0, 1).text = f"{therapist['first_name']} {therapist['last_name']}"
    therapist_table.cell(1, 0).text = "Email:"
    therapist_table.cell(1, 1).text = therapist['email']
    
    doc.add_paragraph()
    
    # Patient Information
    doc.add_heading('Patient Information', level=2)
    patient_table = doc.add_table(rows=8, cols=2)
    patient_table.style = 'Table Grid'
    
    # Calculate age
    dob = datetime.datetime.strptime(patient['date_of_birth'], '%Y-%m-%d').date()
    today = datetime.date.today()
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    
    # Add patient details
    patient_table.cell(0, 0).text = "Name:"
    patient_table.cell(0, 1).text = f"{patient['first_name']} {patient['last_name']}"
    patient_table.cell(4, 0).text = "ID Number:",
    patient_table.cell(1, 0).text = "DOB:"
    patient_table.cell(1, 1).text = datetime.datetime.strptime(patient['date_of_birth'], '%Y-%m-%d').strftime('%B %d, %Y')
    patient_table.cell(2, 0).text = "Age:"
    patient_table.cell(2, 1).text = str(age)
    patient_table.cell(3, 0).text = "Gender:"
    patient_table.cell(3, 1).text = patient['gender'].capitalize() if patient['gender'] else "Not specified"
    patient_table.cell(4, 0).text = "Diagnosis:"
    patient_table.cell(4, 1).text = patient['diagnosis']
    patient_table.cell(5, 0).text = "Medical Aid:"
    patient_table.cell(5, 1).text = patient['medical_aid_name'] or "Not provided"
    patient_table.cell(6, 0).text = "Medical Aid #:"
    patient_table.cell(6, 1).text = patient['medical_aid_number'] or "Not provided"

    doc.add_paragraph()
    
    # Medical History
    doc.add_heading('Medical History', level=2)
    doc.add_paragraph(patient['medical_history'] if patient['medical_history'] else "No medical history recorded.")
    
    doc.add_paragraph()
    
    # Treatment Summary & Progress
    doc.add_heading('Treatment Summary & Progress', level=2)
    
    if not soap_notes:
        doc.add_paragraph("No treatment notes available for this patient.")
    else:
        # Get first and latest note for comparison
        first_note = soap_notes[-1] if soap_notes else None
        latest_note = soap_notes[0] if soap_notes else None
        
        # Initial assessment
        if first_note:
            doc.add_heading('Initial Assessment', level=3)
            first_date = datetime.datetime.strptime(first_note['date'], '%Y-%m-%d %H:%M:%S').strftime('%B %d, %Y')
            doc.add_paragraph(f"Date: {first_date}")
            
            initial_table = doc.add_table(rows=4, cols=2)
            initial_table.style = 'Table Grid'
            
            initial_table.cell(0, 0).text = "Subjective:"
            initial_table.cell(0, 1).text = first_note['subjective']
            initial_table.cell(1, 0).text = "Objective:"
            initial_table.cell(1, 1).text = first_note['objective']
            initial_table.cell(2, 0).text = "Action:"
            initial_table.cell(2, 1).text = first_note['action']
            initial_table.cell(3, 0).text = "Plan:"
            initial_table.cell(3, 1).text = first_note['plan']
            
            doc.add_paragraph()
        
        # Latest assessment (if different from first)
        if latest_note and latest_note['id'] != first_note['id']:
            doc.add_heading('Most Recent Assessment', level=3)
            latest_date = datetime.datetime.strptime(latest_note['date'], '%Y-%m-%d %H:%M:%S').strftime('%B %d, %Y')
            doc.add_paragraph(f"Date: {latest_date}")
            
            latest_table = doc.add_table(rows=4, cols=2)
            latest_table.style = 'Table Grid'
            
            latest_table.cell(0, 0).text = "Subjective:"
            latest_table.cell(0, 1).text = latest_note['subjective']
            latest_table.cell(1, 0).text = "Objective:"
            latest_table.cell(1, 1).text = latest_note['objective']
            latest_table.cell(2, 0).text = "Action:"
            latest_table.cell(2, 1).text = latest_note['action']
            latest_table.cell(3, 0).text = "Plan:"
            latest_table.cell(3, 1).text = latest_note['plan']
            
            doc.add_paragraph()
        
        # Treatment progress
        doc.add_heading('Treatment Progress', level=3)
        if len(soap_notes) > 1:
            progress_paragraph = doc.add_paragraph()
            progress_run = progress_paragraph.add_run(f"Total sessions: {len(soap_notes)}")
            progress_run.bold = True
            
            first_date = datetime.datetime.strptime(soap_notes[-1]['date'], '%Y-%m-%d %H:%M:%S').strftime('%B %d, %Y')
            last_date = datetime.datetime.strptime(soap_notes[0]['date'], '%Y-%m-%d %H:%M:%S').strftime('%B %d, %Y')
            doc.add_paragraph(f"Treatment period: {first_date} to {last_date}")
            
            # Extract goals progress from the most recent note if available
            if latest_note and latest_note['goals_progress']:
                goals_para = doc.add_paragraph()
                goals_run = goals_para.add_run("Goals Progress:")
                goals_run.bold = True
                doc.add_paragraph(latest_note['goals_progress'])
        else:
            doc.add_paragraph("Insufficient data to determine progress. Only one session recorded.")
    
    # Create a buffer to save the file
    buffer = io.BytesIO()
    doc.save(buffer)
    buffer.seek(0)
    
    # Save the file to a location chosen by the user
    file_path = filedialog.asksaveasfilename(
        defaultextension=".docx",
        filetypes=[("Word documents", "*.docx")],
        initialfile=f"Progress_Report_{patient['last_name']}_{patient['first_name']}_{datetime.datetime.now().strftime('%Y%m%d')}.docx"
    )
    
    if file_path:
        with open(file_path, 'wb') as f:
            f.write(buffer.getvalue())
        return {"success": True, "message": f"Report saved to {file_path}"}
    else:
        return {"success": False, "message": "Report generation cancelled"}

# Application UI Class
class EiraNotesApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        
        self.title("Eira Notes - Physiotherapy Management System")
        self.geometry("1200x800")
        self.minsize(800, 600)
       
        # Global state
        self.current_user = None
        self.current_patient = None
        self.current_frame = None
        
        # Configure grid layout
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)
        
        # Create sidebar frame
        self.sidebar_frame = ctk.CTkFrame(self, width=200, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(6, weight=1)
        
        # App logo/title
        self.logo_label = ctk.CTkLabel(self.sidebar_frame, text="Eira Notes", font=ctk.CTkFont(size=20, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        # Sidebar buttons - they will be created after login
        self.sidebar_buttons = []
        
        # Create main content frame
        self.content_frame = ctk.CTkFrame(self, corner_radius=0)
        self.content_frame.grid(row=0, column=1, sticky="nsew", padx=20, pady=20)
        self.content_frame.grid_columnconfigure(0, weight=1)
        self.content_frame.grid_rowconfigure(0, weight=1)

        # Load ICD-10 codes
        with open("icd10.json", encoding="utf8") as f:
            raw = json.load(f)
        self.icd_data = [
            {"code": e["code"], "description": e["description"],
             "lc_code": e["code"].lower(), "lc_desc": e["description"].lower()}
            for e in raw
        ]

        # Show login form initially
        self.show_login()
    
    def clear_content(self):
        # Destroy all widgets in the content frame
        for widget in self.content_frame.winfo_children():
            widget.destroy()
        
        # Reset current frame
        self.current_frame = None
    
    def create_sidebar(self):
        # Clear existing sidebar buttons
        for button in self.sidebar_buttons:
            button.destroy()
        self.sidebar_buttons = []
        
        # Create navigation buttons based on role
        row = 1

        self.home_button = ctk.CTkButton(self.sidebar_frame, text="Home", command=self.show_home)
        self.home_button.grid(row=row, column=0, padx=20, pady=10)
        self.sidebar_buttons.append(self.home_button)
        row += 1

        if self.current_user["role"] in ("admin", "physiotherapist"):
            self.patients_button = ctk.CTkButton(self.sidebar_frame, text="Patients", command=self.show_patients)
            self.patients_button.grid(row=row, column=0, padx=20, pady=10)
            self.sidebar_buttons.append(self.patients_button)
        row += 1
        
        self.appointments_button = ctk.CTkButton(self.sidebar_frame, text="Appointments", command=self.show_appointments)
        self.appointments_button.grid(row=3, column=0, padx=20, pady=10)
        self.sidebar_buttons.append(self.appointments_button)

        row += 1

        if self.current_user["role"] in ("admin", "receptionist"):
            self.billing_button = ctk.CTkButton(self.sidebar_frame, text="Billing", command=self.show_billing)
            self.billing_button.grid(row=row, column=0, padx=20, pady=10)
            self.sidebar_buttons.append(self.billing_button)
        row += 1

        if self.current_user["role"] == "admin":
            self.user_mgmt_button = ctk.CTkButton(self.sidebar_frame, text="User Management", command=self.show_user_management)
            self.user_mgmt_button.grid(row=row, column=0, padx=20, pady=10)
            self.sidebar_buttons.append(self.user_mgmt_button)
        row += 1

        self.reports_button = ctk.CTkButton(self.sidebar_frame, text="Reports", command=self.show_reports)
        self.reports_button.grid(row=4, column=0, padx=20, pady=10)
        self.sidebar_buttons.append(self.reports_button)

        self.billing_button = ctk.CTkButton(self.sidebar_frame, text="Billing", command=self.show_billing)
        self.billing_button.grid(row=5, column=0, padx=20, pady=10)
        self.sidebar_buttons.append(self.billing_button)
        
        # Spacer
        self.sidebar_frame.grid_rowconfigure(row, weight=1)
        
        # User info and logout
        user_name = f"{self.current_user['first_name']} {self.current_user['last_name']}"
        self.user_label = ctk.CTkLabel(self.sidebar_frame, text=user_name, font=ctk.CTkFont(size=12))
        self.user_label.grid(row=row, column=0, padx=20, pady=(10, 0))
        self.sidebar_buttons.append(self.user_label)

        row += 1
        
        self.logout_button = ctk.CTkButton(self.sidebar_frame, text="Logout", fg_color="transparent", command=self.logout)
        self.logout_button.grid(row=row, column=0, padx=20, pady=10)
        self.sidebar_buttons.append(self.logout_button)
    
    def show_login(self):
        self.clear_content()
        
        # Create login frame
        login_frame = ctk.CTkFrame(self.content_frame)
        login_frame.grid(row=0, column=0, sticky="", padx=20, pady=20)
        self.current_frame = login_frame
        
        # Title
        title_label = ctk.CTkLabel(login_frame, text="Login to Eira Notes", font=ctk.CTkFont(size=24, weight="bold"))
        title_label.grid(row=0, column=0, columnspan=2, padx=20, pady=(20, 30))
        
        # Username
        username_label = ctk.CTkLabel(login_frame, text="Username:")
        username_label.grid(row=1, column=0, sticky="e", padx=(20, 10), pady=10)
        username_entry = ctk.CTkEntry(login_frame, width=200)
        username_entry.grid(row=1, column=1, sticky="w", padx=(10, 20), pady=10)
        username_entry.focus()
        
        # Password
        password_label = ctk.CTkLabel(login_frame, text="Password:")
        password_label.grid(row=2, column=0, sticky="e", padx=(20, 10), pady=10)
        password_entry = ctk.CTkEntry(login_frame, width=200, show="•")
        password_entry.grid(row=2, column=1, sticky="w", padx=(10, 20), pady=10)
        
        # Error message
        error_label = ctk.CTkLabel(login_frame, text="", text_color="red")
        error_label.grid(row=3, column=0, columnspan=2, padx=20, pady=(10, 0))
        
        # Login button
        def handle_login():
            username = username_entry.get()
            password = password_entry.get()
            
            if not username or not password:
                error_label.configure(text="Please enter both username and password")
                return
            
            user = authenticate(username, password)
            if user:
                self.current_user = user
                self.create_sidebar()
                self.show_home()
            else:
                error_label.configure(text="Invalid username or password")
        
        login_button = ctk.CTkButton(login_frame, text="Login", command=handle_login)
        login_button.grid(row=4, column=0, columnspan=2, padx=20, pady=(20, 10))
        
        # Register link
        def show_register():
            self.show_register()
        
        register_link = ctk.CTkButton(login_frame, text="Register New Account", fg_color="transparent", command=show_register)
        register_link.grid(row=5, column=0, columnspan=2, padx=20, pady=(10, 20))
    
    def show_register(self):
        self.clear_content()
        
        # Create register frame
        register_frame = ctk.CTkFrame(self.content_frame)
        register_frame.grid(row=0, column=0, sticky="", padx=20, pady=20)
        self.current_frame = register_frame
        
        # Title
        title_label = ctk.CTkLabel(register_frame, text="Register New Account", font=ctk.CTkFont(size=24, weight="bold"))
        title_label.grid(row=0, column=0, columnspan=2, padx=20, pady=(20, 30))
        
        # Username
        username_label = ctk.CTkLabel(register_frame, text="Username:")
        username_label.grid(row=1, column=0, sticky="e", padx=(20, 10), pady=10)
        username_entry = ctk.CTkEntry(register_frame, width=200)
        username_entry.grid(row=1, column=1, sticky="w", padx=(10, 20), pady=10)
        
        # Email
        email_label = ctk.CTkLabel(register_frame, text="Email:")
        email_label.grid(row=2, column=0, sticky="e", padx=(20, 10), pady=10)
        email_entry = ctk.CTkEntry(register_frame, width=200)
        email_entry.grid(row=2, column=1, sticky="w", padx=(10, 20), pady=10)
        
        # First name
        first_name_label = ctk.CTkLabel(register_frame, text="First Name:")
        first_name_label.grid(row=3, column=0, sticky="e", padx=(20, 10), pady=10)
        first_name_entry = ctk.CTkEntry(register_frame, width=200)
        first_name_entry.grid(row=3, column=1, sticky="w", padx=(10, 20), pady=10)
        
        # Last name
        last_name_label = ctk.CTkLabel(register_frame, text="Last Name:")
        last_name_label.grid(row=4, column=0, sticky="e", padx=(20, 10), pady=10)
        last_name_entry = ctk.CTkEntry(register_frame, width=200)
        last_name_entry.grid(row=4, column=1, sticky="w", padx=(10, 20), pady=10)

        # Role Selector
        ctk.CTkLabel(register_frame, text="Role:").grid(row=5, column=0, sticky="e", padx=(20,10), pady=10)
        role_var = ctk.StringVar(value="physiotherapist")
        role_dropdown = ctk.CTkOptionMenu(
            register_frame,
            variable=role_var,
            values=["administrator", "physiotherapist", "receptionist"]
        )
        role_dropdown.grid(row=5, column=1, sticky="w", padx=(10,20), pady=10)
        
        # Password
        password_label = ctk.CTkLabel(register_frame, text="Password:")
        password_label.grid(row=6, column=0, sticky="e", padx=(20, 10), pady=10)
        password_entry = ctk.CTkEntry(register_frame, width=200, show="•")
        password_entry.grid(row=6, column=1, sticky="w", padx=(10, 20), pady=10)
        
        # Confirm password
        confirm_label = ctk.CTkLabel(register_frame, text="Confirm Password:")
        confirm_label.grid(row=7, column=0, sticky="e", padx=(20, 10), pady=10)
        confirm_entry = ctk.CTkEntry(register_frame, width=200, show="•")
        confirm_entry.grid(row=7, column=1, sticky="w", padx=(10, 20), pady=10)
        
        # Error message
        error_label = ctk.CTkLabel(register_frame, text="", text_color="red")
        error_label.grid(row=8, column=0, columnspan=2, padx=20, pady=(10, 0))
        
        # Register button
        def handle_register():
            username = username_entry.get()
            email = email_entry.get()
            first_name = first_name_entry.get()
            last_name = last_name_entry.get()
            password = password_entry.get()
            confirm = confirm_entry.get()
            
            # Simple validation
            if not username or not email or not first_name or not last_name or not password or not confirm:
                error_label.configure(text="All fields are required")
                return
            
            if password != confirm:
                error_label.configure(text="Passwords do not match")
                return
            
            if len(password) < 8:
                error_label.configure(text="Password must be at least 8 characters")
                return
            
            # Register user
            result = register_user(username, email, password, first_name, last_name)
            if result["success"]:
                messagebox.showinfo("Registration Successful", "Your account has been created. You can now log in.")
                self.show_login()
            else:
                error_label.configure(text=result["message"])
        
        register_button = ctk.CTkButton(register_frame, text="Register", command=handle_register)
        register_button.grid(row=8, column=0, columnspan=2, padx=20, pady=(20, 10))
        
        # Back to login
        back_button = ctk.CTkButton(register_frame, text="Back to Login", fg_color="transparent", command=self.show_login)
        back_button.grid(row=9, column=0, columnspan=2, padx=20, pady=(10, 20))
    
    def logout(self):
        self.current_user = None
        
        # Clear sidebar
        for button in self.sidebar_buttons:
            button.destroy()
        self.sidebar_buttons = []
        
        # Show login screen
        self.show_login()
    
    def show_home(self):
        self.clear_content()
        
        # Create home frame
        home_frame = ctk.CTkFrame(self.content_frame)
        home_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        home_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = home_frame
        
        # Welcome message
        welcome_label = ctk.CTkLabel(
            home_frame, 
            text=f"Welcome, {self.current_user['first_name']}!",
            font=ctk.CTkFont(size=24, weight="bold")
        )
        welcome_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        subtitle_label = ctk.CTkLabel(
            home_frame,
            text="Eira Notes - Physiotherapy Practice Management",
            font=ctk.CTkFont(size=16)
        )
        subtitle_label.grid(row=1, column=0, padx=20, pady=(0, 20))
        
        # Quick actions frame
        quick_actions_frame = ctk.CTkFrame(home_frame)
        quick_actions_frame.grid(row=2, column=0, padx=20, pady=20, sticky="ew")
        quick_actions_frame.grid_columnconfigure((0, 1, 2, 3, 4), weight=1)
        
        # Add patient button
        col = 0
        if self.current_user["role"] in ("administrator", "manager", "physiotherapist"):
            add_patient_button = ctk.CTkButton(
                quick_actions_frame,
                text="Add New Patient",
                command=self.show_add_patient,
            )
            add_patient_button.grid(row=0, column=col, padx=10, pady=10)
            col += 1

            view_patients_button = ctk.CTkButton(
                quick_actions_frame,
                text="View All Patients",
                command=self.show_patients,
            )
            view_patients_button.grid(row=0, column=col, padx=10, pady=10)
            col += 1
        
        # Manage appointments button
        manage_appts_button = ctk.CTkButton(
            quick_actions_frame,
            text="Manage Appointments",
            command=self.show_appointments,
        )
        manage_appts_button.grid(row=0, column=col, padx=10, pady=10)
        col += 1

        if self.current_user["role"] in ("administrator", "manager", "receptionist"):
            billing_button = ctk.CTkButton(
                quick_actions_frame,
                text="Billing",
                command=self.show_billing,
            )
            billing_button.grid(row=0, column=col, padx=10, pady=10)

        # Reports button
        reports_button = ctk.CTkButton(
            quick_actions_frame,
            text="Reports",
            command=self.show_reports
        )
        reports_button.grid(row=0, column=3, padx=10, pady=10)

        # Billing button
        billing_button = ctk.CTkButton(
            quick_actions_frame,
            text="Billing",
            command=self.show_billing,
        )
        billing_button.grid(row=0, column=4, padx=10, pady=10)
        
        # Get today's appointments
        appointments = get_appointments(self.current_user["id"], filter_type="today")
        
        # Today's appointments section
        appts_label = ctk.CTkLabel(
            home_frame,
            text="Today's Appointments",
            font=ctk.CTkFont(size=18, weight="bold")
        )
        appts_label.grid(row=3, column=0, padx=20, pady=(20, 10), sticky="w")
        
        # Appointments list
        appointments_frame = ctk.CTkFrame(home_frame)
        appointments_frame.grid(row=4, column=0, padx=20, pady=(0, 20), sticky="ew")
        appointments_frame.grid_columnconfigure(0, weight=1)
        
        if appointments:
            # Create headers
            headers_frame = ctk.CTkFrame(appointments_frame)
            headers_frame.grid(row=0, column=0, padx=5, pady=5, sticky="ew")
            headers_frame.grid_columnconfigure((0, 1, 2, 3), weight=1)
            
            time_header = ctk.CTkLabel(headers_frame, text="Time", font=ctk.CTkFont(weight="bold"))
            time_header.grid(row=0, column=0, padx=5, pady=5, sticky="w")
            
            patient_header = ctk.CTkLabel(headers_frame, text="Patient", font=ctk.CTkFont(weight="bold"))
            patient_header.grid(row=0, column=1, padx=5, pady=5, sticky="w")
            
            duration_header = ctk.CTkLabel(headers_frame, text="Duration", font=ctk.CTkFont(weight="bold"))
            duration_header.grid(row=0, column=2, padx=5, pady=5, sticky="w")
            
            status_header = ctk.CTkLabel(headers_frame, text="Status", font=ctk.CTkFont(weight="bold"))
            status_header.grid(row=0, column=3, padx=5, pady=5, sticky="w")
            
            # Add appointments
            for i, appointment in enumerate(appointments):
                appt_frame = ctk.CTkFrame(appointments_frame)
                appt_frame.grid(row=i+1, column=0, padx=5, pady=2, sticky="ew")
                appt_frame.grid_columnconfigure((0, 1, 2, 3), weight=1)
                
                time_str = datetime.datetime.strptime(appointment["time"], "%H:%M:%S").strftime("%I:%M %p")
                time_label = ctk.CTkLabel(appt_frame, text=time_str)
                time_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
                
                patient_label = ctk.CTkLabel(appt_frame, text=appointment["patient_name"])
                patient_label.grid(row=0, column=1, padx=5, pady=5, sticky="w")
                
                duration_label = ctk.CTkLabel(appt_frame, text=f"{appointment['duration']} min")
                duration_label.grid(row=0, column=2, padx=5, pady=5, sticky="w")
                
                status_label = ctk.CTkLabel(appt_frame, text=appointment["status"].capitalize())
                status_label.grid(row=0, column=3, padx=5, pady=5, sticky="w")
        else:
            no_appt_label = ctk.CTkLabel(appointments_frame, text="No appointments scheduled for today")
            no_appt_label.grid(row=0, column=0, padx=20, pady=20)
    
    def show_patients(self):
        self.clear_content()
        
        # Create patients frame
        patients_frame = ctk.CTkFrame(self.content_frame)
        patients_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        patients_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = patients_frame
        
        # Title and Add button
        header_frame = ctk.CTkFrame(patients_frame)
        header_frame.grid(row=0, column=0, padx=20, pady=20, sticky="ew")
        header_frame.grid_columnconfigure(0, weight=1)
        
        title_label = ctk.CTkLabel(
            header_frame, 
            text="Patients",
            font=ctk.CTkFont(size=24, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=10, sticky="w")
        
        add_button = ctk.CTkButton(
            header_frame,
            text="Add New Patient",
            command=self.show_add_patient
        )
        add_button.grid(row=0, column=1, padx=20, pady=10)
        
        # Get patients
        patients = get_patients(self.current_user["id"])
        
        # Patients list
        patients_list_frame = ctk.CTkFrame(patients_frame)
        patients_list_frame.grid(row=1, column=0, padx=20, pady=(0, 20), sticky="nsew")
        patients_list_frame.grid_columnconfigure(0, weight=1)
        
        # Create a scrollable frame
        scrollable_frame = ctk.CTkScrollableFrame(patients_list_frame, width=800, height=400)
        scrollable_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        scrollable_frame.grid_columnconfigure(0, weight=1)
        
        if patients:
            # Create headers
            headers_frame = ctk.CTkFrame(scrollable_frame)
            headers_frame.grid(row=0, column=0, padx=5, pady=5, sticky="ew")
            headers_frame.grid_columnconfigure((0, 1, 2, 3), weight=1)
            
            name_header = ctk.CTkLabel(headers_frame, text="Patient Name", font=ctk.CTkFont(weight="bold"))
            name_header.grid(row=0, column=0, padx=5, pady=5, sticky="w")
            
            dob_header = ctk.CTkLabel(headers_frame, text="Date of Birth", font=ctk.CTkFont(weight="bold"))
            dob_header.grid(row=0, column=1, padx=5, pady=5, sticky="w")
            
            diagnosis_header = ctk.CTkLabel(headers_frame, text="Diagnosis", font=ctk.CTkFont(weight="bold"))
            diagnosis_header.grid(row=0, column=2, padx=5, pady=5, sticky="w")
            
            actions_header = ctk.CTkLabel(headers_frame, text="Actions", font=ctk.CTkFont(weight="bold"))
            actions_header.grid(row=0, column=3, padx=5, pady=5, sticky="w")
            
            # Add patients
            for i, patient in enumerate(patients):
                patient_frame = ctk.CTkFrame(scrollable_frame)
                patient_frame.grid(row=i+1, column=0, padx=5, pady=2, sticky="ew")
                patient_frame.grid_columnconfigure((0, 1, 2), weight=1)
                
                full_name = f"{patient['first_name']} {patient['last_name']}"
                name_label = ctk.CTkLabel(patient_frame, text=full_name)
                name_label.grid(row=0, column=0, padx=5, pady=10, sticky="w")
                            
                raw_dob = patient.get("date_of_birth", None)
            if raw_dob:
                try:
                    dob = parser.parse(
                        raw_dob, "%Y-%m-%d"
                    ).strftime("%d/%m/%Y")
                except (ValueError, TypeError):
                    dob = raw_dob
            else:
                dob = "Not provided"

                dob_label = ctk.CTkLabel(patient_frame, text=dob)
                dob_label.grid(row=0, column=1, padx=5, pady=10, sticky="w")
                
                diag_label = ctk.CTkLabel(patient_frame, text=patient["diagnosis"])
                diag_label.grid(row=0, column=2, padx=5, pady=10, sticky="w")


                
                # Actions frame
                actions_frame = ctk.CTkFrame(patient_frame)
                actions_frame.grid(row=0, column=3, padx=5, pady=5)
                
                view_button = ctk.CTkButton(
                    actions_frame, 
                    text="View",
                    width=70,
                    command=lambda p=patient: self.show_patient_details(p["id"])
                )
                view_button.grid(row=0, column=0, padx=5, pady=5)
                
                edit_button = ctk.CTkButton(
                    actions_frame, 
                    text="Edit",
                    width=70,
                    command=lambda p=patient: self.show_edit_patient(p["id"])
                )
                edit_button.grid(row=0, column=1, padx=5, pady=5)
        else:
            no_patients_label = ctk.CTkLabel(scrollable_frame, text="No patients found. Add your first patient to get started.")
            no_patients_label.grid(row=0, column=0, padx=20, pady=20)
    
    def show_add_patient(self):
        self.clear_content()
        
        # Create add patient frame
        add_patient_frame = ctk.CTkFrame(self.content_frame)
        add_patient_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        add_patient_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = add_patient_frame
        
        # Title
        title_label = ctk.CTkLabel(
            add_patient_frame, 
            text="Add New Patient",
            font=ctk.CTkFont(size=24, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        # Form
        form_frame = ctk.CTkFrame(add_patient_frame)
        form_frame.grid(row=1, column=0, padx=20, pady=20, sticky="nsew")
        form_frame.grid_columnconfigure((0, 1), weight=1)
        
        # Create a scrollable frame for the form
        scrollable_frame = ctk.CTkScrollableFrame(form_frame, width=800, height=400)
        scrollable_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        scrollable_frame.grid_columnconfigure((0, 1), weight=1)
        
        # Personal Information Section
        personal_label = ctk.CTkLabel(
            scrollable_frame, 
            text="Personal Information",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        personal_label.grid(row=0, column=0, columnspan=2, padx=10, pady=(10, 5), sticky="w")
        
        # First Name
        first_name_label = ctk.CTkLabel(scrollable_frame, text="First Name:")
        first_name_label.grid(row=1, column=0, padx=10, pady=5, sticky="e")
        first_name_entry = ctk.CTkEntry(scrollable_frame, width=250)
        first_name_entry.grid(row=1, column=1, padx=10, pady=5, sticky="w")
        
        # Last Name
        last_name_label = ctk.CTkLabel(scrollable_frame, text="Last Name:")
        last_name_label.grid(row=2, column=0, padx=10, pady=5, sticky="e")
        last_name_entry = ctk.CTkEntry(scrollable_frame, width=250)
        last_name_entry.grid(row=2, column=1, padx=10, pady=5, sticky="w")
        
        # Date of Birth
        dob_label = ctk.CTkLabel(scrollable_frame, text="Date of Birth:")
        dob_label.grid(row=3, column=0, padx=10, pady=5, sticky="e")
        
        dob_frame = ctk.CTkFrame(scrollable_frame)
        dob_frame.grid(row=3, column=1, padx=10, pady=5, sticky="w")
        
        # Create dropdown for day, month, year
        day_var = tk.StringVar()
        day_dropdown = ctk.CTkOptionMenu(dob_frame, variable=day_var, values=[str(i).zfill(2) for i in range(1, 32)])
        day_dropdown.grid(row=0, column=0, padx=2)
        day_dropdown.set("01")
        
        month_var = tk.StringVar()
        month_dropdown = ctk.CTkOptionMenu(dob_frame, variable=month_var, values=[str(i).zfill(2) for i in range(1, 13)])
        month_dropdown.grid(row=0, column=1, padx=2)
        month_dropdown.set("01")
        
        current_year = datetime.datetime.now().year
        year_var = tk.StringVar()
        year_dropdown = ctk.CTkOptionMenu(dob_frame, variable=year_var, values=[str(i) for i in range(current_year-100, current_year+1)])
        year_dropdown.grid(row=0, column=2, padx=2)
        year_dropdown.set(str(current_year-30))  # Default to 30 years ago
        
        # Gender
        gender_label = ctk.CTkLabel(scrollable_frame, text="Gender:")
        gender_label.grid(row=4, column=0, padx=10, pady=5, sticky="e")
        
        gender_var = tk.StringVar()
        gender_dropdown = ctk.CTkOptionMenu(scrollable_frame, variable=gender_var, values=["male", "female", "other"])
        gender_dropdown.grid(row=4, column=1, padx=10, pady=5, sticky="w")
        gender_dropdown.set("Select Gender")

        # Id number
        id_label = ctk.CTkLabel(scrollable_frame, text="ID Number:")
        id_label.grid(row=5, column=0, padx=10, pady=5, sticky="e")
        id_entry = ctk.CTkEntry(scrollable_frame, width=250)
        id_entry.grid(row=5, column=1, padx=10, pady=5, sticky="w")

        # Contact Information Section
        contact_label = ctk.CTkLabel(
            scrollable_frame, 
            text="Contact Information",
            font=ctk.CTkFont(size=14, weight="bold")
        )
        contact_label.grid(row=6, column=0, columnspan=2, padx=10, pady=(20, 5), sticky="w")
        
        # Phone
        phone_label = ctk.CTkLabel(scrollable_frame, text="Phone Number:")
        phone_label.grid(row=7, column=0, padx=10, pady=5, sticky="e")
        phone_entry = ctk.CTkEntry(scrollable_frame, width=250)
        phone_entry.grid(row=7, column=1, padx=10, pady=5, sticky="w")

        # Second Phone
        phone2_label = ctk.CTkLabel(scrollable_frame, text="Second Phone:")
        phone2_label.grid(row=8, column=0, padx=10, pady=5, sticky="e")
        phone2_entry = ctk.CTkEntry(scrollable_frame, width=250)
        phone2_entry.grid(row=8, column=1, padx=10, pady=5, sticky="w")
        
        # Email
        email_label = ctk.CTkLabel(scrollable_frame, text="Email:")
        email_label.grid(row=9, column=0, padx=10, pady=5, sticky="e")
        email_entry = ctk.CTkEntry(scrollable_frame, width=250)
        email_entry.grid(row=9, column=1, padx=10, pady=5, sticky="w")
        
        # Address
        address_label = ctk.CTkLabel(scrollable_frame, text="Address:")
        address_label.grid(row=10, column=0, padx=10, pady=5, sticky="e")
        address_entry = ctk.CTkEntry(scrollable_frame, width=250)
        address_entry.grid(row=10, column=1, padx=10, pady=5, sticky="w")

        # P.O Box
        pobox_label = ctk.CTkLabel(scrollable_frame, text="P.O Box:")
        pobox_label.grid(row=11, column=0, padx=10, pady=5, sticky="e")
        pobox_entry = ctk.CTkEntry(scrollable_frame, width=250)
        pobox_entry.grid(row=11, column=1, padx=10, pady=5, sticky="w")
        
        # Medical Information Section
        medical_label = ctk.CTkLabel(
            scrollable_frame, 
            text="Medical Information",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        medical_label.grid(row=12, column=0, columnspan=2, padx=10, pady=(20, 5), sticky="w")
        
        # Medical Aid
        med_aid_label = ctk.CTkLabel(scrollable_frame, text="Medical Aid Provider:")
        med_aid_label.grid(row=13, column=0, padx=10, pady=5, sticky="e")
        med_aid_entry = ctk.CTkEntry(scrollable_frame, width=250)
        med_aid_entry.grid(row=13, column=1, padx=10, pady=5, sticky="w")
        
        # Medical Aid Number
        med_num_label = ctk.CTkLabel(scrollable_frame, text="Medical Aid Number:")
        med_num_label.grid(row=14, column=0, padx=10, pady=5, sticky="e")
        med_num_entry = ctk.CTkEntry(scrollable_frame, width=250)
        med_num_entry.grid(row=14, column=1, padx=10, pady=5, sticky="w")
        # Alias for backward compatibility with older notebooks/scripts
        id_number = med_num_entry

        # Medical Aid ID
        med_id_label = ctk.CTkLabel(scrollable_frame, text="Medical Aid ID:")
        med_id_label.grid(row=15, column=0, padx=10, pady=5, sticky="e")
        med_id_entry = ctk.CTkEntry(scrollable_frame, width=250)
        med_id_entry.grid(row=15, column=1, padx=10, pady=5, sticky="w")
        
        # Medical History
        history_label = ctk.CTkLabel(scrollable_frame, text="Medical History:")
        history_label.grid(row=16, column=0, padx=10, pady=5, sticky="ne")
        history_entry = ctk.CTkTextbox(scrollable_frame, width=250, height=100)
        history_entry.grid(row=16, column=1, padx=10, pady=5, sticky="w")
        
        # Diagnosis
        diagnosis_label = ctk.CTkLabel(scrollable_frame, text="Diagnosis:")
        diagnosis_label.grid(row=17, column=0, padx=10, pady=5, sticky="e")
        diagnosis_entry = ctk.CTkEntry(scrollable_frame, width=250)
        diagnosis_entry.grid(row=17, column=1, padx=10, pady=5, sticky="w")
        self.diagnosis_entry = diagnosis_entry
        icd_code_label = ctk.CTkLabel(scrollable_frame, text="ICD Code:")
        icd_code_label.grid(row=18, column=0, padx=10, pady=5, sticky="e")
        icd_code_entry = ctk.CTkEntry(scrollable_frame, width=250)
        icd_code_entry.grid(row=18, column=1, padx=10, pady=5, sticky="w")
        self.icd_code_entry = icd_code_entry
        self.icd_suggestions = tk.Listbox(scrollable_frame, height=6)
        self.icd_suggestions.grid(row=19, column=1, padx=10, sticky="ew")
        self.icd_suggestions.bind("<<ListboxSelect>>", self.on_icd_select)
        self.icd_suggestions.lower()
        diagnosis_entry.bind("<KeyRelease>", self.on_diagnosis_type)

     
        # Error message
        error_label = ctk.CTkLabel(scrollable_frame, text="", text_color="red")
        error_label.grid(row=20, column=0, columnspan=2, padx=10, pady=(10, 0))
        
        
        # Save and Cancel buttons
        button_frame = ctk.CTkFrame(scrollable_frame)
        button_frame.grid(row=21, column=0, columnspan=2, padx=10, pady=20)
        
        def handle_save():
            # Get form data
            first_name = first_name_entry.get()
            last_name = last_name_entry.get()
            date_of_birth = f"{year_var.get()}-{month_var.get()}-{day_var.get()}"
            gender = gender_var.get() if gender_var.get() != "Select Gender" else ""
            
            # Validate required fields
            if not first_name or not last_name or gender == "Select Gender":
                error_label.configure(text="First name, last name, and gender are required")
                return
            
            try:
                datetime.datetime.strptime(date_of_birth, "%Y-%m-%d")
            except ValueError:
                error_label.configure(text="Invalid date of birth")
                return
            
            if not diagnosis_entry.get():
                error_label.configure(text="Diagnosis is required")
                return
            
            # Collect all data
            patient_data = {
                "first_name": first_name,
                "last_name": last_name,
                "id_number": id_number.get(),
                "date_of_birth": date_of_birth,
                "gender": gender,
                "phone": phone_entry.get(),
                "phone2": phone2_entry.get(),
                "email": email_entry.get(),
                "address": address_entry.get(),
                "po_box": pobox_entry.get(),
                "medical_aid_name": med_aid_entry.get(),
                "medical_aid_number": med_num_entry.get(),
                "medical_aid_id": med_id_entry.get(),
                "medical_history": history_entry.get("1.0", "end-1c"),
                "diagnosis": diagnosis_entry.get(),
                "icd_code": icd_code_entry.get(),
            }
            
            # Add patient
            result = add_patient(patient_data, self.current_user["id"])
            if result["success"]:
                messagebox.showinfo("Success", result["message"])
                self.show_patients()
            else:
                error_label.configure(text=result["message"])
        
        save_button = ctk.CTkButton(
            button_frame,
            text="Save Patient",
            command=handle_save
        )
        save_button.grid(row=0, column=0, padx=10, pady=10)
        
        cancel_button = ctk.CTkButton(
            button_frame,
            text="Cancel",
            fg_color="transparent",
            command=self.show_patients
        )
        cancel_button.grid(row=0, column=1, padx=10, pady=10)

    def suggest_icd(self, query, limit=10):
        q = query.strip().lower()
        if not q:
            return []
        matches = []
        for e in self.icd_data:
            if q in e["lc_desc"] or e["lc_code"].startswith(q):
                matches.append(e)
                if len(matches) >= limit:
                    break
        return matches

    def on_diagnosis_type(self, event):
        text = event.widget.get()
        matches = self.suggest_icd(text)
        lb = self.icd_suggestions
        lb.delete(0, tk.END)
        if not matches:
            lb.lower()
            return
        for e in matches:
            lb.insert(tk.END, f"{e['code']}: {e['description']}")
        lb.lift()

    def on_icd_select(self, event):
        sel = event.widget.curselection()
        if not sel:
            return
        line = event.widget.get(sel[0])
        code, desc = line.split(":", 1)
        self.icd_code_entry.delete(0, tk.END)
        self.icd_code_entry.insert(0, code.strip())
        self.diagnosis_entry.delete(0, tk.END)
        self.diagnosis_entry.insert(0, desc.strip())
        event.widget.delete(0, tk.END)
        event.widget.lower()
        
    def show_patient_details(self, patient_id):
        self.clear_content()
        
        # Get patient data
        conn = sqlite3.connect(DB_PATH)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
        patient = dict(cursor.fetchone())
        
        # Get patient age
        dob = parser.parse(patient['date_of_birth']).date()
        today = datetime.date.today()
        age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

        def open_report(path):
            return  # temporarily disable opening files
            if sys.platform.startswith('win'):
                os.startfile(path)
            elif sys.platform == 'darwin':
                subprocess.call(['open', path])
            else:
                subprocess.call(['xdg-open', path])
        
        # Create patient details frame
        details_frame = ctk.CTkFrame(self.content_frame)
        details_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        details_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = details_frame
        
        # Header with patient name and action buttons
        header_frame = ctk.CTkFrame(details_frame)
        header_frame.grid(row=0, column=0, padx=20, pady=20, sticky="ew")
        header_frame.grid_columnconfigure(0, weight=1)
        
        name = f"{patient['first_name']} {patient['last_name']}"
        title_label = ctk.CTkLabel(
            header_frame, 
            text=name,
            font=ctk.CTkFont(size=24, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=10, sticky="w")
        
        # Buttons for actions
        buttons_frame = ctk.CTkFrame(header_frame)
        buttons_frame.grid(row=0, column=1, padx=20, pady=10)
        
        edit_button = ctk.CTkButton(
            buttons_frame,
            text="Edit Patient",
            command=lambda: self.show_edit_patient(patient_id)
        )
        edit_button.grid(row=0, column=0, padx=5, pady=5)
        
        delete_button = ctk.CTkButton(
            buttons_frame,
            text="Delete Patient",
            fg_color="red",
            hover_color="#990000",
            command=lambda: self.delete_patient_confirm(patient_id)
        )
        delete_button.grid(row=0, column=1, padx=5, pady=5)
        
        # Create tabview for different sections
        tabview = ctk.CTkTabview(details_frame)
        tabview.grid(row=1, column=0, padx=20, pady=20, sticky="nsew")
        
        # Add tabs
        info_tab = tabview.add("Patient Info")
        notes_tab = tabview.add("SOAP Notes")
        appointments_tab = tabview.add("Appointments")
        reports_tab = tabview.add("Reports")
        
        # Configure tab grids
        info_tab.grid_columnconfigure(0, weight=1)
        notes_tab.grid_columnconfigure(0, weight=1)
        appointments_tab.grid_columnconfigure(0, weight=1)
        reports_tab.grid_columnconfigure(0, weight=1)
        
        # Patient Info Tab
        info_frame = ctk.CTkFrame(info_tab)
        info_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        info_frame.grid_columnconfigure((0, 1), weight=1)
        
        # Personal info section
        personal_label = ctk.CTkLabel(
            info_frame, 
            text="Personal Information",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        personal_label.grid(row=0, column=0, columnspan=2, padx=10, pady=(10, 5), sticky="w")
        
        # Create two columns for info
        left_info = ctk.CTkFrame(info_frame)
        left_info.grid(row=1, column=0, padx=10, pady=5, sticky="nsew")
        
        right_info = ctk.CTkFrame(info_frame)
        right_info.grid(row=1, column=1, padx=10, pady=5, sticky="nsew")
        
        # Left column info
        dob_str = datetime.datetime.strptime(patient['date_of_birth'], '%Y-%m-%d').strftime('%d %B %Y')
        dob_raw = parser.parse(patient['date_of_birth']).strftime('%d %B %Y')
        try:
            dob_str = datetime.datetime.strptime(dob_raw, '%Y-%m-%d').strftime('%d %B %Y')
        except ValueError:
            dob_str = dob_raw or "Invalid date"
        
        ctk.CTkLabel(left_info, text="Age:", font=ctk.CTkFont(weight="bold")).grid(row=0, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(left_info, text=str(age)).grid(row=0, column=1, padx=10, pady=5, sticky="w")
        
        ctk.CTkLabel(left_info, text="Date of Birth:", font=ctk.CTkFont(weight="bold")).grid(row=1, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(left_info, text=dob_str).grid(row=1, column=1, padx=10, pady=5, sticky="w")
        
        ctk.CTkLabel(left_info, text="Gender:", font=ctk.CTkFont(weight="bold")).grid(row=2, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(left_info, text=patient['gender'].capitalize() if patient['gender'] else "Not specified").grid(row=2, column=1, padx=10, pady=5, sticky="w")

        ctk.CTkLabel(left_info, text="ID Number:", font=ctk.CTkFont(weight="bold")).grid(row=3, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(left_info, text=patient['id_number'] or "Not provided").grid(row=3, column=1, padx=10, pady=5, sticky="w")

        # Contact info
        ctk.CTkLabel(right_info, text="Phone:", font=ctk.CTkFont(weight="bold")).grid(row=0, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(right_info, text=patient['phone'] or "Not provided").grid(row=0, column=1, padx=10, pady=5, sticky="w")
        
        ctk.CTkLabel(right_info, text="Second Phone:", font=ctk.CTkFont(weight="bold")).grid(row=1, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(right_info, text=patient.get('phone2') or "Not provided").grid(row=1, column=1, padx=10, pady=5, sticky="w")

        ctk.CTkLabel(right_info, text="Email:", font=ctk.CTkFont(weight="bold")).grid(row=2, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(right_info, text=patient['email'] or "Not provided").grid(row=2, column=1, padx=10, pady=5, sticky="w")

        ctk.CTkLabel(right_info, text="Address:", font=ctk.CTkFont(weight="bold")).grid(row=3, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(right_info, text=patient['address'] or "Not provided").grid(row=3, column=1, padx=10, pady=5, sticky="w")

        ctk.CTkLabel(right_info, text="P.O. Box:", font=ctk.CTkFont(weight="bold")).grid(row=4, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(right_info, text=patient.get('po_box') or "Not provided").grid(row=4, column=1, padx=10, pady=5, sticky="w")
        
        # Medical info section
        medical_label = ctk.CTkLabel(
            info_frame, 
            text="Medical Information",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        medical_label.grid(row=2, column=0, columnspan=2, padx=10, pady=(20, 5), sticky="w")
        
        med_info = ctk.CTkFrame(info_frame)
        med_info.grid(row=3, column=0, columnspan=2, padx=10, pady=5, sticky="nsew")
        
        ctk.CTkLabel(med_info, text="Medical Aid:", font=ctk.CTkFont(weight="bold")).grid(row=0, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(med_info, text=patient['medical_aid_name'] or "Not provided").grid(row=0, column=1, padx=10, pady=5, sticky="w")
        
        ctk.CTkLabel(med_info, text="Medical Aid Number:", font=ctk.CTkFont(weight="bold")).grid(row=1, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(med_info, text=patient['medical_aid_number'] or "Not provided").grid(row=1, column=1, padx=10, pady=5, sticky="w")

        ctk.CTkLabel(med_info, text="Medical Aid ID:", font=ctk.CTkFont(weight="bold")).grid(row=2, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(med_info, text=patient.get('medical_aid_id') or "Not provided").grid(row=2, column=1, padx=10, pady=5, sticky="w")
        
        ctk.CTkLabel(med_info, text="Diagnosis:", font=ctk.CTkFont(weight="bold")).grid(row=3, column=0, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(med_info, text=patient['diagnosis']).grid(row=3, column=1, padx=10, pady=5, sticky="w")
        
        # Medical history
        history_label = ctk.CTkLabel(
            info_frame, 
            text="Medical History",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        history_label.grid(row=4, column=0, columnspan=2, padx=10, pady=(20, 5), sticky="w")
        
        history_text = ctk.CTkTextbox(info_frame, height=100, wrap="word")
        history_text.grid(row=5, column=0, columnspan=2, padx=10, pady=5, sticky="nsew")
        history_text.insert("1.0", patient['medical_history'] or "No medical history recorded.")
        history_text.configure(state="disabled")  # Make read-only
        
        
        # SOAP Notes Tab
        notes_container = ctk.CTkFrame(notes_tab)
        notes_container.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        notes_container.grid_columnconfigure(0, weight=1)
        
        # Header with add button
        notes_header = ctk.CTkFrame(notes_container)
        notes_header.grid(row=0, column=0, padx=5, pady=5, sticky="ew")
        notes_header.grid_columnconfigure(0, weight=1)
        
        notes_title = ctk.CTkLabel(
            notes_header, 
            text="SOAP Notes",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        notes_title.grid(row=0, column=0, padx=10, pady=10, sticky="w")
        
        add_note_button = ctk.CTkButton(
            notes_header,
            text="Add New SOAP Note",
            command=lambda: self.show_add_soap_note(patient_id)
        )
        add_note_button.grid(row=0, column=1, padx=10, pady=10)
        
        # Get soap notes
        soap_notes = get_soap_notes(patient_id)
        
        # Notes list in scrollable frame
        notes_scrollable = ctk.CTkScrollableFrame(notes_container, height=300)
        notes_scrollable.grid(row=1, column=0, padx=5, pady=5, sticky="nsew")
        notes_scrollable.grid_columnconfigure(0, weight=1)
        
        if soap_notes:
            for i, note in enumerate(soap_notes):
                note_frame = ctk.CTkFrame(notes_scrollable)
                note_frame.grid(row=i, column=0, padx=5, pady=5, sticky="ew")
                note_frame.grid_columnconfigure(1, weight=1)
                
                date_str = datetime.datetime.strptime(note['date'], '%Y-%m-%d').strftime('%d %B %Y')
                
                date_label = ctk.CTkLabel(note_frame, text=date_str, font=ctk.CTkFont(weight="bold"))
                date_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")
                
                # Add a brief excerpt of the subjective field
                excerpt = note['subjective'][:50] + "..." if len(note['subjective']) > 50 else note['subjective']
                excerpt_label = ctk.CTkLabel(note_frame, text=excerpt)
                excerpt_label.grid(row=0, column=1, padx=10, pady=10, sticky="w")
                
                view_button = ctk.CTkButton(
                    note_frame,
                    text="View",
                    width=70,
                    command=lambda n=note: self.show_soap_note_details(n['id'], patient_id)
                )
                view_button.grid(row=0, column=2, padx=10, pady=10)
        else:
            no_notes_label = ctk.CTkLabel(notes_scrollable, text="No SOAP notes recorded for this patient.")
            no_notes_label.grid(row=0, column=0, padx=20, pady=20)

        # Reports Tab
        reports_container = ctk.CTkFrame(reports_tab)
        reports_container.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        reports_container.grid_columnconfigure(0, weight=1)
        reports_header = ctk.CTkFrame(reports_container)
        reports_header.grid(row=0, column=0, padx=5, pady=5, sticky="ew")
        reports_header.grid_columnconfigure(0, weight=1)
        reports_title = ctk.CTkLabel(
            reports_header, 
            text="Reports",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        reports_title.grid(row=0, column=0, padx=10, pady=10, sticky="w")
        reports = open_report(patient_id)
        reports_scrollable = ctk.CTkScrollableFrame(reports_container, height=300)
        reports_scrollable.grid(row=1, column=0, padx=5, pady=5, sticky="nsew")
        reports_scrollable.grid_columnconfigure(0, weight=1)
        if reports:
            for i, report in enumerate(reports):
                rep_frame = ctk.CTkFrame(reports_scrollable)
                rep_frame.grid(row=i, column=0, padx=5, pady=5, sticky="ew")
                rep_frame.grid_columnconfigure((0, 1), weight=1)
                date_str = datetime.datetime.strptime(report['created_at'], '%Y-%m-%d %H:%M:%S').strftime('%d %B %Y')
                date_label = ctk.CTkLabel(rep_frame, text=date_str, font=ctk.CTkFont(weight="bold"))
                date_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")
                name_label = ctk.CTkLabel(rep_frame, text=os.path.basename(report['file_path']))
                name_label.grid(row=0, column=1, padx=10, pady=10, sticky="w")
                open_button = ctk.CTkButton(rep_frame, text="Open", width=70, command=lambda p=report['file_path']: open_report(p))
                open_button.grid(row=0, column=2, padx=10, pady=10)
        else:
            no_reports_label = ctk.CTkLabel(reports_scrollable, text="No reports recorded for this patient.")
            no_reports_label.grid(row=0, column=0, padx=20, pady=20)

        
        # Appointments Tab
        appts_container = ctk.CTkFrame(appointments_tab)
        appts_container.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        appts_container.grid_columnconfigure(0, weight=1)
        
        # Header with add button
        appts_header = ctk.CTkFrame(appts_container)
        appts_header.grid(row=0, column=0, padx=5, pady=5, sticky="ew")
        appts_header.grid_columnconfigure(0, weight=1)
        
        appts_title = ctk.CTkLabel(
            appts_header, 
            text="Appointments",
            font=ctk.CTkFont(size=16, weight="bold")
        )
        appts_title.grid(row=0, column=0, padx=10, pady=10, sticky="w")
        
        add_appt_button = ctk.CTkButton(
            appts_header,
            text="Schedule New Appointment",
            command=lambda: self.show_add_appointment(patient_id)
        )
        add_appt_button.grid(row=0, column=1, padx=10, pady=10)
        
        # Get appointments for this patient
        cursor.execute(
            """SELECT * FROM appointments 
            WHERE patient_id = ? 
            ORDER BY date DESC, time DESC""",
            (patient_id,)
        )
        appointments = [dict(row) for row in cursor.fetchall()]
        conn.close()
        
        # Appointments list in scrollable frame
        appts_scrollable = ctk.CTkScrollableFrame(appts_container, height=300)
        appts_scrollable.grid(row=1, column=0, padx=5, pady=5, sticky="nsew")
        appts_scrollable.grid_columnconfigure(0, weight=1)
        
        if appointments:
            # Table headers
            headers_frame = ctk.CTkFrame(appts_scrollable)
            headers_frame.grid(row=0, column=0, padx=5, pady=5, sticky="ew")
            headers_frame.grid_columnconfigure((0, 1, 2, 3), weight=1)
            
            date_header = ctk.CTkLabel(headers_frame, text="Date", font=ctk.CTkFont(weight="bold"))
            date_header.grid(row=0, column=0, padx=5, pady=5, sticky="w")
            
            time_header = ctk.CTkLabel(headers_frame, text="Time", font=ctk.CTkFont(weight="bold"))
            time_header.grid(row=0, column=1, padx=5, pady=5, sticky="w")
            
            duration_header = ctk.CTkLabel(headers_frame, text="Duration", font=ctk.CTkFont(weight="bold"))
            duration_header.grid(row=0, column=2, padx=5, pady=5, sticky="w")
            
            status_header = ctk.CTkLabel(headers_frame, text="Status", font=ctk.CTkFont(weight="bold"))
            status_header.grid(row=0, column=3, padx=5, pady=5, sticky="w")
            
            actions_header = ctk.CTkLabel(headers_frame, text="Actions", font=ctk.CTkFont(weight="bold"))
            actions_header.grid(row=0, column=4, padx=5, pady=5, sticky="w")
            
            # List appointments
            for i, appt in enumerate(appointments):
                appt_frame = ctk.CTkFrame(appts_scrollable)
                appt_frame.grid(row=i+1, column=0, padx=5, pady=2, sticky="ew")
                appt_frame.grid_columnconfigure((0, 1, 2, 3), weight=1)

            try:
                time_str = datetime.datetime.strptime(appointment["time"], "%H:%M:%S").strftime("%I:%M %p")
            except ValueError:
                time_str = appointment["time"]  # Display raw string if parsing fails

                date_str = datetime.datetime.strptime(appt['date'], '%Y-%m-%d').strftime('%d/%m/%Y')
                date_label = ctk.CTkLabel(appt_frame, text=date_str)
                date_label.grid(row=0, column=0, padx=5, pady=10, sticky="w")
                
                time_str = datetime.datetime.strptime(appt['time'], '%H:%M:%S').strftime('%I:%M %p')
                time_label = ctk.CTkLabel(appt_frame, text=time_str)
                time_label.grid(row=0, column=1, padx=5, pady=10, sticky="w")
                
                duration_label = ctk.CTkLabel(appt_frame, text=f"{appt['duration']} min")
                duration_label.grid(row=0, column=2, padx=5, pady=10, sticky="w")
                
                status_label = ctk.CTkLabel(appt_frame, text=appt['status'].capitalize())
                status_label.grid(row=0, column=3, padx=5, pady=10, sticky="w")
                
                # Action buttons
                action_frame = ctk.CTkFrame(appt_frame)
                action_frame.grid(row=0, column=4, padx=5, pady=5)
                
                edit_appt_button = ctk.CTkButton(
                    action_frame,
                    text="Edit",
                    width=70,
                    command=lambda a=appt: self.show_edit_appointment(a['id'], patient_id)
                )
                edit_appt_button.grid(row=0, column=0, padx=5, pady=5)
                
                delete_appt_button = ctk.CTkButton(
                    action_frame,
                    text="Cancel",
                    width=70,
                    fg_color="red",
                    hover_color="#990000",
                    command=lambda a=appt: self.delete_appointment_confirm(a['id'], patient_id)
                )
                delete_appt_button.grid(row=0, column=1, padx=5, pady=5)
        else:
            no_appts_label = ctk.CTkLabel(appts_scrollable, text="No appointments scheduled for this patient.")
            no_appts_label.grid(row=0, column=0, padx=20, pady=20)
       
        # Back button at the bottom
        back_button = ctk.CTkButton(
            details_frame,
            text="Back to Patients",
            command=self.show_patients
        )
        back_button.grid(row=2, column=0, padx=20, pady=20)
        
    def show_edit_patient(self, patient_id):
        self.clear_content()

        conn = sqlite3.connect(DB_PATH)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
        patient = dict(cursor.fetchone())
        conn.close()

        frame = ctk.CTkFrame(self.content_frame)
        frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        frame.grid_columnconfigure(0, weight=1)
        self.current_frame = frame

        title = ctk.CTkLabel(
            frame,
            text="Edit Patient",
            font=ctk.CTkFont(size=24, weight="bold")
        )
        title.grid(row=0, column=0, padx=20, pady=(20,10))

        form = ctk.CTkFrame(frame)
        form.grid(row=1, column=0, padx=20, pady=20, sticky="nsew")
        form.grid_columnconfigure((0,1), weight=1)
        scroll = ctk.CTkScrollableFrame(form, width=800, height=400)
        scroll.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        scroll.grid_columnconfigure((0,1), weight=1)

        # Personal info
        ctk.CTkLabel(scroll, text="Personal Information", font=ctk.CTkFont(size=16, weight="bold")).grid(row=0, column=0, columnspan=2, padx=10, pady=(10,5), sticky="w")

        ctk.CTkLabel(scroll, text="First Name:").grid(row=1, column=0, padx=10, pady=5, sticky="e")
        first_name_entry = ctk.CTkEntry(scroll, width=250)
        first_name_entry.grid(row=1, column=1, padx=10, pady=5, sticky="w")
        first_name_entry.insert(0, patient['first_name'])

        ctk.CTkLabel(scroll, text="Last Name:").grid(row=2, column=0, padx=10, pady=5, sticky="e")
        last_name_entry = ctk.CTkEntry(scroll, width=250)
        last_name_entry.grid(row=2, column=1, padx=10, pady=5, sticky="w")
        last_name_entry.insert(0, patient['last_name'])

        # Date of birth fields
        dob_frame = ctk.CTkFrame(scroll)
        dob_frame.grid(row=3, column=1, padx=10, pady=5, sticky="w")
        dob = datetime.datetime.parser.parse(patient['date_of_birth'], '%Y-%m-%d')
        day_var = tk.StringVar(value=dob.strftime('%d'))
        ctk.CTkOptionMenu(dob_frame, variable=day_var, values=[str(i).zfill(2) for i in range(1,32)]).grid(row=0,column=0,padx=2)
        month_var = tk.StringVar(value=dob.strftime('%m'))
        ctk.CTkOptionMenu(dob_frame, variable=month_var, values=[str(i).zfill(2) for i in range(1,13)]).grid(row=0,column=1,padx=2)
        year_var = tk.StringVar(value=dob.strftime('%Y'))
        current_year = datetime.datetime.now().year
        ctk.CTkOptionMenu(dob_frame, variable=year_var, values=[str(i) for i in range(current_year-100, current_year+1)]).grid(row=0,column=2,padx=2)
        ctk.CTkLabel(scroll, text="Date of Birth:").grid(row=3, column=0, padx=10, pady=5, sticky="e")

        gender_var = tk.StringVar(value=patient['gender'] if patient['gender'] else "Select Gender")
        ctk.CTkLabel(scroll, text="Gender:").grid(row=4, column=0, padx=10, pady=5, sticky="e")
        gender_dropdown = ctk.CTkOptionMenu(scroll, variable=gender_var, values=["male","female","other"])
        gender_dropdown.grid(row=4, column=1, padx=10, pady=5, sticky="w")

        # Contact info
        ctk.CTkLabel(scroll, text="Contact Information", font=ctk.CTkFont(size=16, weight="bold")).grid(row=5, column=0, columnspan=2, padx=10, pady=(20,5), sticky="w")
        
        ctk.CTkLabel(scroll, text="Phone Number:").grid(row=6, column=0, padx=10, pady=5, sticky="e")
        phone_entry = ctk.CTkEntry(scroll, width=250)
        phone_entry.grid(row=6, column=1, padx=10, pady=5, sticky="w")
        phone_entry.insert(0, patient.get('phone',''))
        
        ctk.CTkLabel(scrollable_frame, text="Second Phone:").grid(row=7, column=0, padx=10, pady=5, sticky="e")
        phone2_entry = ctk.CTkEntry(scrollable_frame, width=250)
        phone2_entry.grid(row=7, column=1, padx=10, pady=5, sticky="w")
        phone2_entry.insert(0, patient.get('phone2',''))
        
        ctk.CTkLabel(scroll, text="Email:").grid(row=8, column=0, padx=10, pady=5, sticky="e")
        email_entry = ctk.CTkEntry(scroll, width=250)
        email_entry.grid(row=8, column=1, padx=10, pady=5, sticky="w")
        email_entry.insert(0, patient.get('email',''))
        
        ctk.CTkLabel(scroll, text="Address:").grid(row=9, column=0, padx=10, pady=5, sticky="e")
        address_entry = ctk.CTkEntry(scroll, width=250)
        address_entry.grid(row=9, column=1, padx=10, pady=5, sticky="w")
        address_entry.insert(0, patient.get('address',''))

        ctk.CTkLabel(scroll, text="P.O Box:").grid(row=10, column=0, padx=10, pady=5, sticky="e")
        po_box_label = ctk.CTkLabel(scrollable_frame, text="P.O. Box:")
        po_box_label.grid(row=10, column=0, padx=10, pady=5, sticky="e")
        po_box_entry = ctk.CTkEntry(scrollable_frame, width=250)
        po_box_entry.grid(row=10, column=1, padx=10, pady=5, sticky="w")


        # Medical info
        ctk.CTkLabel(scroll, text="Medical Information", font=ctk.CTkFont(size=16, weight="bold")).grid(row=9, column=0, columnspan=2, padx=10, pady=(20,5), sticky="w")
        
        ctk.CTkLabel(scroll, text="Medical Aid Provider:").grid(row=11, column=0, padx=10, pady=5, sticky="e")
        med_aid_entry = ctk.CTkEntry(scroll, width=250)
        med_aid_entry.grid(row=11, column=1, padx=10, pady=5, sticky="w")
        med_aid_entry.insert(0, patient.get('medical_aid_name',''))
        
        ctk.CTkLabel(scroll, text="Medical Aid Number:").grid(row=12, column=0, padx=10, pady=5, sticky="e")
        med_num_entry = ctk.CTkEntry(scroll, width=250)
        med_num_entry.grid(row=12, column=1, padx=10, pady=5, sticky="w")
        med_num_entry.insert(0, patient.get('medical_aid_number',''))
        
        ctk.CTkLabel(scrollable_frame, text="Medical Aid ID:").grid(row=13, column=0, padx=10, pady=5, sticky="e")
        med_id_entry = ctk.CTkEntry(scrollable_frame, width=250)
        med_id_entry.grid(row=13, column=1, padx=10, pady=5, sticky="w")
        med_id_entry.insert(0, patient.get('medical_aid_id',''))
        
        ctk.CTkLabel(scroll, text="Medical History:").grid(row=14, column=0, padx=10, pady=5, sticky="ne")
        history_entry = ctk.CTkTextbox(scroll, width=250, height=100)
        history_entry.grid(row=14, column=1, padx=10, pady=5, sticky="w")
        history_entry.insert("1.0", patient.get('medical_history',''))

        ctk.CTkLabel(scroll, text="Diagnosis:").grid(row=15, column=0, padx=10, pady=5, sticky="e")
        diagnosis_entry = ctk.CTkEntry(scroll, width=250)
        diagnosis_entry.grid(row=15, column=1, padx=10, pady=5, sticky="w")
        diagnosis_entry.insert(0, patient.get('diagnosis',''))

        error_label = ctk.CTkLabel(scroll, text="", text_color="red")
        error_label.grid(row=16, column=0, columnspan=2, padx=10, pady=(10,0))

        button_frame = ctk.CTkFrame(scroll)
        button_frame.grid(row=16, column=0, columnspan=2, padx=10, pady=20)

        def handle_save():
            first_name = first_name_entry.get()
            last_name = last_name_entry.get()
            gender = gender_var.get() if gender_var.get() != "Select Gender" else ""
            date_of_birth = f"{year_var.get()}-{month_var.get()}-{day_var.get()}"

            if not first_name or not last_name or gender == "":
                error_label.configure(text="First name, last name, and gender are required")
                return
            try:
                datetime.datetime.strptime(date_of_birth, '%Y-%m-%d')
            except ValueError:
                error_label.configure(text="Invalid date of birth")
                return
            if not diagnosis_entry.get():
                error_label.configure(text="Diagnosis is required")
                return

            data = {
                "first_name": first_name,
                "last_name": last_name,
                "id_number": id_number_entry.get(),
                "date_of_birth": date_of_birth,
                "gender": gender,
                "phone": phone_entry.get(),
                "email": email_entry.get(),
                "address": address_entry.get(),
                "medical_aid_name": med_aid_entry.get(),
                "medical_aid_number": med_num_entry.get(),
                "medical_history": history_entry.get("1.0", "end-1c"),
                "diagnosis": diagnosis_entry.get()
            }

            result = update_patient(patient_id, data)
            if result["success"]:
                messagebox.showinfo("Success", result["message"])
                self.show_patient_details(patient_id)
            else:
                error_label.configure(text=result["message"])

        save_button = ctk.CTkButton(button_frame, text="Update Patient", command=handle_save)
        save_button.grid(row=0, column=0, padx=10, pady=10)
        cancel_button = ctk.CTkButton(button_frame, text="Cancel", fg_color="transparent", command=lambda: self.show_patient_details(patient_id))
        cancel_button.grid(row=0, column=1, padx=10, pady=10)
    
    def delete_patient_confirm(self, patient_id):
        # Confirm deletion
        if messagebox.askyesno("Confirm Delete", "Are you sure you want to delete this patient? This will also delete all associated SOAP notes and appointments.\n\nThis action cannot be undone."):
            result = delete_patient(patient_id)
            if result["success"]:
                messagebox.showinfo("Success", result["message"])
                self.show_patients()
            else:
                messagebox.showerror("Error", result["message"])
    
    def show_add_soap_note(self, patient_id):
        self.clear_content()
        
        # Get patient info
        conn = sqlite3.connect(DB_PATH)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
        patient = dict(cursor.fetchone())
        conn.close()
        
        # Create soap note frame
        soap_frame = ctk.CTkFrame(self.content_frame)
        soap_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        soap_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = soap_frame
        
        # Header
        title_label = ctk.CTkLabel(
            soap_frame, 
            text=f"New SOAP Note for {patient['first_name']} {patient['last_name']}",
            font=ctk.CTkFont(size=20, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        # Form in scrollable frame
        form_frame = ctk.CTkScrollableFrame(soap_frame, height=500)
        form_frame.grid(row=1, column=0, padx=20, pady=10, sticky="nsew")
        form_frame.grid_columnconfigure(0, weight=1)
        
        # Date field
        date_frame = ctk.CTkFrame(form_frame)
        date_frame.grid(row=0, column=0, padx=10, pady=10, sticky="ew")
        date_frame.grid_columnconfigure(1, weight=1)
        
        date_label = ctk.CTkLabel(date_frame, text="Date:")
        date_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")
        
        date_picker_frame = ctk.CTkFrame(date_frame)
        date_picker_frame.grid(row=0, column=1, padx=10, pady=10, sticky="w")
        
        today = datetime.datetime.now()
        
        # Create dropdown for day, month, year
        day_var = tk.StringVar(value=today.strftime("%d"))
        day_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=day_var, values=[str(i).zfill(2) for i in range(1, 32)])
        day_dropdown.grid(row=0, column=0, padx=2)
        
        month_var = tk.StringVar(value=today.strftime("%m"))
        month_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=month_var, values=[str(i).zfill(2) for i in range(1, 13)])
        month_dropdown.grid(row=0, column=1, padx=2)
        
        year_var = tk.StringVar(value=today.strftime("%Y"))
        year_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=year_var, values=[str(i) for i in range(today.year-5, today.year+1)])
        year_dropdown.grid(row=0, column=2, padx=2)
        
        # SOAP components
        soap_components = [
            {"label": "Subjective (Patient Report)", "field": "subjective", "placeholder": "Patient's self-reported symptoms, concerns, progress, etc."},
            {"label": "Objective (Measurements & Observations)", "field": "objective", "placeholder": "Clinical observations, measurements, test results, etc."},
            {"label": "Action (Treatment/Management)", "field": "action", "placeholder": "Treatment provided, management performed, therapeutic interventions, etc."},
            {"label": "Plan (Treatment Plan)", "field": "plan", "placeholder": "Treatment plan, goals, recommended exercises, follow-up schedule, etc."}
        ]
        
        soap_textboxes = {}
        
        for i, component in enumerate(soap_components):
            component_frame = ctk.CTkFrame(form_frame)
            component_frame.grid(row=i+1, column=0, padx=10, pady=10, sticky="ew")
            component_frame.grid_columnconfigure(0, weight=1)
            
            label = ctk.CTkLabel(component_frame, text=component["label"], font=ctk.CTkFont(weight="bold"))
            label.grid(row=0, column=0, padx=10, pady=5, sticky="w")
            
            textbox = ctk.CTkTextbox(component_frame, height=100, width=600)
            textbox.grid(row=1, column=0, padx=10, pady=5, sticky="ew")
            textbox.insert("1.0", component["placeholder"])
            textbox.bind("<FocusIn>", lambda e, tb=textbox, ph=component["placeholder"]: self.clear_placeholder(tb, ph))
            textbox.bind("<FocusOut>", lambda e, tb=textbox, ph=component["placeholder"]: self.restore_placeholder(tb, ph))
            
            soap_textboxes[component["field"]] = textbox
        
        # Additional fields
        additional_components = [
            {"label": "Treatment Provided", "field": "treatment_provided", "placeholder": "Specific treatments delivered during this session"},
            {"label": "Patient Response", "field": "patient_response", "placeholder": "How the patient responded to treatment"},
            {"label": "Goals & Progress", "field": "goals_progress", "placeholder": "Progress toward established goals, new goals set"}
        ]
        
        additional_textboxes = {}
        
        for i, component in enumerate(additional_components):
            component_frame = ctk.CTkFrame(form_frame)
            component_frame.grid(row=i+len(soap_components)+1, column=0, padx=10, pady=10, sticky="ew")
            component_frame.grid_columnconfigure(0, weight=1)
            
            label = ctk.CTkLabel(component_frame, text=component["label"], font=ctk.CTkFont(weight="bold"))
            label.grid(row=0, column=0, padx=10, pady=5, sticky="w")
            
            textbox = ctk.CTkTextbox(component_frame, height=80, width=600)
            textbox.grid(row=1, column=0, padx=10, pady=5, sticky="ew")
            textbox.insert("1.0", component["placeholder"])
            textbox.bind("<FocusIn>", lambda e, tb=textbox, ph=component["placeholder"]: self.clear_placeholder(tb, ph))
            textbox.bind("<FocusOut>", lambda e, tb=textbox, ph=component["placeholder"]: self.restore_placeholder(tb, ph))
            
            additional_textboxes[component["field"]] = textbox
        
        # Error message
        error_label = ctk.CTkLabel(form_frame, text="", text_color="red")
        error_label.grid(row=len(soap_components)+len(additional_components)+1, column=0, padx=10, pady=(10, 0))
        
        # Save and Cancel buttons
        button_frame = ctk.CTkFrame(form_frame)
        button_frame.grid(row=17, column=0, columnspan=2, padx=10, pady=20)
        
        def handle_save():
            # Validate required fields
            for field, textbox in soap_textboxes.items():
                text = textbox.get("1.0", "end-1c")
                if text == "" or text == soap_components[next(i for i, c in enumerate(soap_components) if c["field"] == field)]["placeholder"]:
                    error_label.configure(text=f"Please complete the {field.capitalize()} field")
                    return
            
            # Get form data
            date_str = f"{year_var.get()}-{month_var.get()}-{day_var.get()}"
            
            soap_data = {
                "date": date_str,
                "subjective": soap_textboxes["subjective"].get("1.0", "end-1c"),
                "objective": soap_textboxes["objective"].get("1.0", "end-1c"),
                "action": soap_textboxes["action"].get("1.0", "end-1c"),
                "plan": soap_textboxes["plan"].get("1.0", "end-1c")
            }
            
            # Add additional fields if they're not placeholders
            for field, textbox in additional_textboxes.items():
                text = textbox.get("1.0", "end-1c")
                placeholder = next(c["placeholder"] for c in additional_components if c["field"] == field)
                if text != placeholder:
                    soap_data[field] = text
            
            # Add SOAP note
            result = add_soap_note(soap_data, patient_id, self.current_user["id"])
            if result["success"]:
                messagebox.showinfo("Success", result["message"])
                self.show_patient_details(patient_id)
            else:
                error_label.configure(text=result["message"])
        
        save_button = ctk.CTkButton(
            button_frame,
            text="Save SOAP Note",
            command=handle_save
        )
        save_button.grid(row=0, column=0, padx=10, pady=10)
        
        cancel_button = ctk.CTkButton(
            button_frame,
            text="Cancel",
            fg_color="transparent",
            command=lambda: self.show_patient_details(patient_id)
        )
        cancel_button.grid(row=0, column=1, padx=10, pady=10)
    
    def clear_placeholder(self, textbox, placeholder):
        if textbox.get("1.0", "end-1c") == placeholder:
            textbox.delete("1.0", "end")
    
    def restore_placeholder(self, textbox, placeholder):
        if textbox.get("1.0", "end-1c").strip() == "":
            textbox.delete("1.0", "end")
            textbox.insert("1.0", placeholder)
    
    def show_soap_note_details(self, note_id, patient_id):
        self.clear_content()
        
        # Get note data
        conn = sqlite3.connect(DB_PATH)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        cursor.execute("SELECT * FROM soap_notes WHERE id = ?", (note_id,))
        note = dict(cursor.fetchone())
        
        cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
        patient = dict(cursor.fetchone())
        
        conn.close()
        
        # Create note details frame
        note_frame = ctk.CTkFrame(self.content_frame)
        note_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        note_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = note_frame
        
        # Header with date and action buttons
        header_frame = ctk.CTkFrame(note_frame)
        header_frame.grid(row=0, column=0, padx=20, pady=20, sticky="ew")
        header_frame.grid_columnconfigure(0, weight=1)
        
        date_str = datetime.datetime.parser.parse(note['date'], '%Y-%m-%d').strftime('%d %B %Y')
        title_label = ctk.CTkLabel(
            header_frame, 
            text=f"SOAP Note: {date_str}",
            font=ctk.CTkFont(size=20, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=10, sticky="w")
        
        subtitle_label = ctk.CTkLabel(
            header_frame, 
            text=f"Patient: {patient['first_name']} {patient['last_name']}",
            font=ctk.CTkFont(size=14)
        )
        subtitle_label.grid(row=1, column=0, padx=20, pady=(0, 10), sticky="w")
        
        # Buttons for actions
        buttons_frame = ctk.CTkFrame(header_frame)
        buttons_frame.grid(row=0, column=1, rowspan=2, padx=20, pady=10)
        
        if note["created_by"] == self.current_user["id"]:
            edit_button = ctk.CTkButton(
                buttons_frame,
                text="Edit Note",
                command=lambda: self.show_edit_soap_note(note_id, patient_id)
            )
            edit_button.grid(row=0, column=0, padx=5, pady=5)

            delete_button = ctk.CTkButton(
                buttons_frame,
                text="Delete Note",
                fg_color="red",
                hover_color="#990000",
                command=lambda: self.delete_soap_note_confirm(note_id, patient_id)
            )
            delete_button.grid(row=0, column=1, padx=5, pady=5)
        
        # Create scrollable frame for note content
        note_content = ctk.CTkScrollableFrame(note_frame, height=400)
        note_content.grid(row=1, column=0, padx=20, pady=10, sticky="nsew")
        note_content.grid_columnconfigure(0, weight=1)
        
        # SOAP Components
        soap_components = [
            {"label": "Subjective", "field": "subjective", "color": "#3B8ED0"},  # Blue
            {"label": "Objective", "field": "objective", "color": "#1F6AA5"},     # Darker blue
            {"label": "Action", "field": "action", "color": "#2FA572"},          # Green
            {"label": "Plan", "field": "plan", "color": "#F0B86E"}               # Yellow/orange
        ]
        
        # Display SOAP components in a 2x2 grid
        grid_frame = ctk.CTkFrame(note_content)
        grid_frame.grid(row=0, column=0, padx=10, pady=10, sticky="ew")
        grid_frame.grid_columnconfigure((0, 1), weight=1)
        
        for i, component in enumerate(soap_components):
            row, col = divmod(i, 2)
            
            component_frame = ctk.CTkFrame(grid_frame)
            component_frame.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")
            component_frame.grid_columnconfigure(0, weight=1)
            component_frame.grid_rowconfigure(1, weight=1)
            
            header = ctk.CTkFrame(component_frame, fg_color=component["color"])
            header.grid(row=0, column=0, sticky="ew")
            
            label = ctk.CTkLabel(header, text=component["label"], font=ctk.CTkFont(weight="bold"), text_color="white")
            label.grid(row=0, column=0, padx=10, pady=5)
            
            text_box = ctk.CTkTextbox(component_frame, height=150, wrap="word", state="normal")
            text_box.grid(row=1, column=0, padx=0, pady=0, sticky="nsew")
            text_box.insert("1.0", note[component["field"]])
            text_box.configure(state="disabled")  # Make read-only
        
        # Additional fields if they exist
        additional_fields = [
            {"label": "Treatment Provided", "field": "treatment_provided"},
            {"label": "Patient Response", "field": "patient_response"},
            {"label": "Goals & Progress", "field": "goals_progress"}
        ]
        
        # Check if any additional fields have content
        has_additional = any(note.get(field["field"]) for field in additional_fields)
        
        if has_additional:
            additional_label = ctk.CTkLabel(
                note_content, 
                text="Additional Information",
                font=ctk.CTkFont(size=16, weight="bold")
            )
            additional_label.grid(row=1, column=0, padx=10, pady=(20, 10), sticky="w")
            
            additional_frame = ctk.CTkFrame(note_content)
            additional_frame.grid(row=2, column=0, padx=10, pady=10, sticky="ew")
            additional_frame.grid_columnconfigure(0, weight=1)
            
            row = 0
            for field in additional_fields:
                if note.get(field["field"]):
                    field_frame = ctk.CTkFrame(additional_frame)
                    field_frame.grid(row=row, column=0, padx=10, pady=5, sticky="ew")
                    field_frame.grid_columnconfigure(0, weight=1)
                    
                    field_label = ctk.CTkLabel(field_frame, text=field["label"], font=ctk.CTkFont(weight="bold"))
                    field_label.grid(row=0, column=0, padx=10, pady=5, sticky="w")
                    
                    field_text = ctk.CTkTextbox(field_frame, height=80, wrap="word", state="normal")
                    field_text.grid(row=1, column=0, padx=10, pady=5, sticky="ew")
                    field_text.insert("1.0", note[field["field"]])
                    field_text.configure(state="disabled")  # Make read-only
                    
                    row += 1
        
        # Back button
        back_button = ctk.CTkButton(
            note_frame,
            text="Back to Patient",
            command=lambda: self.show_patient_details(patient_id)
        )
        back_button.grid(row=2, column=0, padx=20, pady=20)

    def show_edit_soap_note(self, note_id, patient_id):
        self.clear_content()

        conn = sqlite3.connect(DB_PATH)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM soap_notes WHERE id = ?", (note_id,))
        note = dict(cursor.fetchone())
        cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
        patient = dict(cursor.fetchone())
        conn.close()

        soap_frame = ctk.CTkFrame(self.content_frame)
        soap_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        soap_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = soap_frame

        title_label = ctk.CTkLabel(
            soap_frame,
            text=f"Edit SOAP Note for {patient['first_name']} {patient['last_name']}",
            font=ctk.CTkFont(size=20, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        form_frame = ctk.CTkScrollableFrame(soap_frame, height=500)
        form_frame.grid(row=1, column=0, padx=20, pady=10, sticky="nsew")
        form_frame.grid_columnconfigure(0, weight=1)

        date_frame = ctk.CTkFrame(form_frame)
        date_frame.grid(row=0, column=0, padx=10, pady=10, sticky="ew")
        date_frame.grid_columnconfigure(1, weight=1)

        date_label = ctk.CTkLabel(date_frame, text="Date:")
        date_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")

        date_picker_frame = ctk.CTkFrame(date_frame)
        date_picker_frame.grid(row=0, column=1, padx=10, pady=10, sticky="w")

        note_date = datetime.datetime.strptime(note['date'], '%Y-%m-%d')
        day_var = tk.StringVar(value=note_date.strftime('%d'))
        day_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=day_var, values=[str(i).zfill(2) for i in range(1, 32)])
        day_dropdown.grid(row=0, column=0, padx=2)

        month_var = tk.StringVar(value=note_date.strftime('%m'))
        month_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=month_var, values=[str(i).zfill(2) for i in range(1, 13)])
        month_dropdown.grid(row=0, column=1, padx=2)

        year_var = tk.StringVar(value=note_date.strftime('%Y'))
        year_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=year_var, values=[str(i) for i in range(note_date.year-5, note_date.year+1)])
        year_dropdown.grid(row=0, column=2, padx=2)

        soap_components = [
            {"label": "Subjective (Patient Report)", "field": "subjective"},
            {"label": "Objective (Measurements & Observations)", "field": "objective"},
            {"label": "Action (Treatment/Management)", "field": "action"},
            {"label": "Plan (Treatment Plan)", "field": "plan"}
        ]

        soap_textboxes = {}
        for i, comp in enumerate(soap_components):
            component_frame = ctk.CTkFrame(form_frame)
            component_frame.grid(row=i+1, column=0, padx=10, pady=10, sticky="ew")
            component_frame.grid_columnconfigure(0, weight=1)

            label = ctk.CTkLabel(component_frame, text=comp["label"], font=ctk.CTkFont(weight="bold"))
            label.grid(row=0, column=0, padx=10, pady=5, sticky="w")

            textbox = ctk.CTkTextbox(component_frame, height=100, width=600)
            textbox.grid(row=1, column=0, padx=10, pady=5, sticky="ew")
            textbox.insert("1.0", note[comp["field"]] or "")
            soap_textboxes[comp["field"]] = textbox

        additional_components = [
            {"label": "Treatment Provided", "field": "treatment_provided"},
            {"label": "Patient Response", "field": "patient_response"},
            {"label": "Goals & Progress", "field": "goals_progress"}
        ]

        additional_textboxes = {}
        for i, comp in enumerate(additional_components):
            component_frame = ctk.CTkFrame(form_frame)
            component_frame.grid(row=i+len(soap_components)+1, column=0, padx=10, pady=10, sticky="ew")
            component_frame.grid_columnconfigure(0, weight=1)

            label = ctk.CTkLabel(component_frame, text=comp["label"], font=ctk.CTkFont(weight="bold"))
            label.grid(row=0, column=0, padx=10, pady=5, sticky="w")

            textbox = ctk.CTkTextbox(component_frame, height=80, width=600)
            textbox.grid(row=1, column=0, padx=10, pady=5, sticky="ew")
            if note.get(comp["field"]):
                textbox.insert("1.0", note[comp["field"]])
            additional_textboxes[comp["field"]] = textbox

        error_label = ctk.CTkLabel(form_frame, text="", text_color="red")
        error_label.grid(row=len(soap_components)+len(additional_components)+1, column=0, padx=10, pady=(10, 0))

        button_frame = ctk.CTkFrame(form_frame)
        button_frame.grid(row=len(soap_components)+len(additional_components)+2, column=0, padx=10, pady=20)

        def handle_save():
            for field, textbox in soap_textboxes.items():
                if textbox.get('1.0', 'end-1c').strip() == '':
                    error_label.configure(text=f"Please complete the {field.capitalize()} field")
                    return

            date_str = f"{year_var.get()}-{month_var.get()}-{day_var.get()}"

            soap_data = {
                'date': date_str,
                'subjective': soap_textboxes['subjective'].get('1.0', 'end-1c'),
                'objective': soap_textboxes['objective'].get('1.0', 'end-1c'),
                'action': soap_textboxes['action'].get('1.0', 'end-1c'),
                'plan': soap_textboxes['plan'].get('1.0', 'end-1c')
            }

            for field, textbox in additional_textboxes.items():
                text = textbox.get('1.0', 'end-1c').strip()
                if text:
                    soap_data[field] = text

            result = update_soap_note(note_id, soap_data)
            if result['success']:
                messagebox.showinfo('Success', result['message'])
                self.show_soap_note_details(note_id, patient_id)
            else:
                error_label.configure(text=result['message'])

        save_button = ctk.CTkButton(button_frame, text="Update SOAP Note", command=handle_save)
        save_button.grid(row=0, column=0, padx=10, pady=10)

        cancel_button = ctk.CTkButton(
            button_frame,
            text="Cancel",
            fg_color="transparent",
            command=lambda: self.show_soap_note_details(note_id, patient_id)
        )
        cancel_button.grid(row=0, column=1, padx=10, pady=10)
    
    def delete_soap_note_confirm(self, note_id, patient_id):
        # Confirm deletion
        if messagebox.askyesno("Confirm Delete", "Are you sure you want to delete this SOAP note?\n\nThis action cannot be undone."):
            result = delete_soap_note(note_id, self.current_user["id"])
            if result["success"]:
                messagebox.showinfo("Success", result["message"])
                self.show_patient_details(patient_id)
            else:
                messagebox.showerror("Error", result["message"])
    
    def show_add_appointment(self, patient_id):
        self.clear_content()
        
        # Get patient info
        conn = sqlite3.connect(DB_PATH)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
        patient = dict(cursor.fetchone())
        conn.close()
        
        # Create appointment frame
        appt_frame = ctk.CTkFrame(self.content_frame)
        appt_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        appt_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = appt_frame
        
        # Header
        title_label = ctk.CTkLabel(
            appt_frame, 
            text=f"Schedule Appointment for {patient['first_name']} {patient['last_name']}",
            font=ctk.CTkFont(size=20, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        # Form
        form_frame = ctk.CTkFrame(appt_frame)
        form_frame.grid(row=1, column=0, padx=20, pady=20, sticky="nsew")
        form_frame.grid_columnconfigure((0, 1), weight=1)
        
        # Date
        date_label = ctk.CTkLabel(form_frame, text="Date:")
        date_label.grid(row=0, column=0, padx=10, pady=10, sticky="e")
        
        date_picker_frame = ctk.CTkFrame(form_frame)
        date_picker_frame.grid(row=0, column=1, padx=10, pady=10, sticky="w")
        
        today = datetime.datetime.now()
        
        # Create dropdown for day, month, year
        day_var = tk.StringVar(value=today.strftime("%d"))
        day_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=day_var, values=[str(i).zfill(2) for i in range(1, 32)])
        day_dropdown.grid(row=0, column=0, padx=2)
        
        month_var = tk.StringVar(value=today.strftime("%m"))
        month_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=month_var, values=[str(i).zfill(2) for i in range(1, 13)])
        month_dropdown.grid(row=0, column=1, padx=2)
        
        year_var = tk.StringVar(value=today.strftime("%Y"))
        year_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=year_var, values=[str(i) for i in range(today.year, today.year+3)])
        year_dropdown.grid(row=0, column=2, padx=2)
        
        # Time
        time_label = ctk.CTkLabel(form_frame, text="Time:")
        time_label.grid(row=1, column=0, padx=10, pady=10, sticky="e")
        
        times = ["09:20", "10:00", "10:40", "11:20", "12:00", "12:40", "13:20", "14:00", "14:40", "15:20", "16:00"]
        time_var = tk.StringVar(value=times[0])
        time_dropdown = ctk.CTkOptionMenu(form_frame, variable=time_var, values=times)
        time_dropdown.grid(row=1, column=1, padx=10, pady=10, sticky="w")
       
        # Duration
        duration_label = ctk.CTkLabel(form_frame, text="Duration (minutes):")
        duration_label.grid(row=2, column=0, padx=10, pady=10, sticky="e")
        
        duration_var = tk.StringVar(value="40")
        duration_dropdown = ctk.CTkOptionMenu(form_frame, variable=duration_var, values=["40"])
        duration_dropdown.grid(row=2, column=1, padx=10, pady=10, sticky="w")
        
        # Status
        status_label = ctk.CTkLabel(form_frame, text="Status:")
        status_label.grid(row=3, column=0, padx=10, pady=10, sticky="e")
        
        status_var = tk.StringVar(value="scheduled")
        status_dropdown = ctk.CTkOptionMenu(form_frame, variable=status_var, values=["scheduled", "completed", "cancelled"])
        status_dropdown.grid(row=3, column=1, padx=10, pady=10, sticky="w")
        
        # Notes
        notes_label = ctk.CTkLabel(form_frame, text="Notes:")
        notes_label.grid(row=4, column=0, padx=10, pady=10, sticky="ne")
        
        notes_textbox = ctk.CTkTextbox(form_frame, height=100, width=300)
        notes_textbox.grid(row=4, column=1, padx=10, pady=10, sticky="w")
        
        # Error message
        error_label = ctk.CTkLabel(form_frame, text="", text_color="red")
        error_label.grid(row=5, column=0, columnspan=2, padx=10, pady=(10, 0))
        
        # Buttons
        button_frame = ctk.CTkFrame(form_frame)
        button_frame.grid(row=6, column=0, columnspan=2, padx=10, pady=20)
        
        def handle_save():
            try:
                # Validate date
                date_str = f"{year_var.get()}-{month_var.get()}-{day_var.get()}"
                datetime.datetime.strptime(date_str, "%Y-%m-%d")
                
                # Create time string
                time_str = f"{time_var.get()}:00"
                
                # Validate duration
                duration = int(duration_var.get())
                
                # Create appointment data
                appt_data = {
                    "date": date_str,
                    "time": time_str,
                    "duration": duration,
                    "status": status_var.get(),
                    "notes": notes_textbox.get("1.0", "end-1c") if notes_textbox.get("1.0", "end-1c").strip() else None
                }
                
                # Add appointment
                result = add_appointment(appt_data, patient_id, self.current_user["id"])
                if result["success"]:
                    messagebox.showinfo("Success", result["message"])
                    self.show_patient_details(patient_id)
                else:
                    error_label.configure(text=result["message"])
            except ValueError:
                error_label.configure(text="Invalid date format")
        
        save_button = ctk.CTkButton(
            button_frame,
            text="Save Appointment",
            command=handle_save
        )
        save_button.grid(row=0, column=0, padx=10, pady=10)
        
        cancel_button = ctk.CTkButton(
            button_frame,
            text="Cancel",
            fg_color="transparent",
            command=lambda: self.show_patient_details(patient_id)
        )
        cancel_button.grid(row=0, column=1, padx=10, pady=10)
    
    def delete_appointment_confirm(self, appointment_id, patient_id):
        # Confirm deletion
        if messagebox.askyesno("Confirm Cancel", "Are you sure you want to cancel this appointment?\n\nThis action cannot be undone."):
            result = delete_appointment(appointment_id)
            if result["success"]:
                messagebox.showinfo("Success", result["message"])
                self.show_patient_details(patient_id)
            else:
                messagebox.showerror("Error", result["message"])
    
    def show_appointments(self):
        self.clear_content()
        
        # Create appointments frame
        appointments_frame = ctk.CTkFrame(self.content_frame)
        appointments_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        appointments_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = appointments_frame
        
        # Title 
        title_label = ctk.CTkLabel(
            appointments_frame, 
            text="Appointments",
            font=ctk.CTkFont(size=24, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        # Filter options
        filter_frame = ctk.CTkFrame(appointments_frame)
        filter_frame.grid(row=1, column=0, padx=20, pady=10, sticky="ew")
        
        filter_label = ctk.CTkLabel(filter_frame, text="Filter:")
        filter_label.grid(row=0, column=0, padx=10, pady=10)
        
        # Create filter buttons that act like tabs
        filter_options = [
            {"value": None, "text": "All Appointments"},
            {"value": "today", "text": "Today"},
            {"value": "upcoming", "text": "Upcoming"}
        ]
        
        filter_buttons = []
        current_filter = [None]  # Use a list to create a mutable reference
        
        for i, option in enumerate(filter_options):
            button = ctk.CTkButton(
                filter_frame,
                text=option["text"],
                fg_color="transparent" if option["value"] != current_filter[0] else None,
                command=lambda val=option["value"], btn_idx=i: self.filter_appointments(val, btn_idx, filter_buttons, current_filter)
            )
            button.grid(row=0, column=i+1, padx=10, pady=10)
            filter_buttons.append(button)
        
        # Appointments container
        self.appointments_container = ctk.CTkFrame(appointments_frame)
        self.appointments_container.grid(row=2, column=0, padx=20, pady=10, sticky="nsew")
        self.appointments_container.grid_columnconfigure(0, weight=1)
        
        # Load initial appointments (all)
        self.load_appointments(None)
    
    def filter_appointments(self, filter_value, button_idx, filter_buttons, current_filter):
        # Update button styles
        for i, button in enumerate(filter_buttons):
            if i == button_idx:
                button.configure(fg_color=("gray75", "gray25"))  # Selected
            else:
                button.configure(fg_color="transparent")  # Unselected
        
        # Update current filter
        current_filter[0] = filter_value
        
        # Load appointments with the selected filter
        self.load_appointments(filter_value)

    def show_edit_appointment(self, appointment_id, patient_id):
        self.clear_content()

        conn = sqlite3.connect(DB_PATH)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM appointments WHERE id = ?", (appointment_id,))
        appt = dict(cursor.fetchone())
        cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
        patient = dict(cursor.fetchone())
        conn.close()

        appt_frame = ctk.CTkFrame(self.content_frame)
        appt_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        appt_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = appt_frame

        title_label = ctk.CTkLabel(
            appt_frame,
            text=f"Edit Appointment for {patient['first_name']} {patient['last_name']}",
            font=ctk.CTkFont(size=20, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        form_frame = ctk.CTkFrame(appt_frame)
        form_frame.grid(row=1, column=0, padx=20, pady=20, sticky="nsew")
        form_frame.grid_columnconfigure((0, 1), weight=1)

        # Date
        date_label = ctk.CTkLabel(form_frame, text="Date:")
        date_label.grid(row=0, column=0, padx=10, pady=10, sticky="e")

        date_picker_frame = ctk.CTkFrame(form_frame)
        date_picker_frame.grid(row=0, column=1, padx=10, pady=10, sticky="w")

        today = datetime.datetime.now()
        
        # Create dropdown for day, month, year
        day_var = tk.StringVar(value=today.strftime("%d"))
        day_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=day_var, values=[str(i).zfill(2) for i in range(1, 32)])
        day_dropdown.grid(row=0, column=0, padx=2)
        
        month_var = tk.StringVar(value=today.strftime("%m"))
        month_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=month_var, values=[str(i).zfill(2) for i in range(1, 13)])
        month_dropdown.grid(row=0, column=1, padx=2)
        
        year_var = tk.StringVar(value=today.strftime("%Y"))
        year_dropdown = ctk.CTkOptionMenu(date_picker_frame, variable=year_var, values=[str(i) for i in range(today.year, today.year+3)])
        year_dropdown.grid(row=0, column=2, padx=2)
        

        # Time
        time_label = ctk.CTkLabel(form_frame, text="Time:")
        time_label.grid(row=1, column=0, padx=10, pady=10, sticky="e")

        times = ["09:20", "10:00", "10:40", "11:20", "12:00", "12:40", "13:20", "14:00", "14:40", "15:20", "16:00"]
        time_var = tk.StringVar(value=times[0])
        time_dropdown = ctk.CTkOptionMenu(form_frame, variable=time_var, values=times)
        time_dropdown.grid(row=1, column=1, padx=10, pady=10, sticky="w")

        # Duration
        duration_label = ctk.CTkLabel(form_frame, text="Duration (minutes):")
        duration_label.grid(row=2, column=0, padx=10, pady=10, sticky="e")

        duration_var = tk.StringVar(value="40")
        duration_dropdown = ctk.CTkOptionMenu(form_frame, variable=duration_var, values=["40"])
        duration_dropdown.grid(row=2, column=1, padx=10, pady=10, sticky="w")

        # Status
        status_label = ctk.CTkLabel(form_frame, text="Status:")
        status_label.grid(row=3, column=0, padx=10, pady=10, sticky="e")

        status_var = tk.StringVar(value=appt['status'])
        status_dropdown = ctk.CTkOptionMenu(
            form_frame,
            variable=status_var,
            values=["scheduled", "completed", "cancelled"]
        )
        status_dropdown.grid(row=3, column=1, padx=10, pady=10, sticky="w")

        # Notes
        notes_label = ctk.CTkLabel(form_frame, text="Notes:")
        notes_label.grid(row=4, column=0, padx=10, pady=10, sticky="ne")

        notes_textbox = ctk.CTkTextbox(form_frame, height=100, width=300)
        notes_textbox.grid(row=4, column=1, padx=10, pady=10, sticky="w")
        notes_textbox.insert("1.0", appt['notes'] if appt['notes'] else "")

        error_label = ctk.CTkLabel(form_frame, text="", text_color="red")
        error_label.grid(row=17, column=0, columnspan=2, padx=10, pady=(10, 0))

        button_frame = ctk.CTkFrame(form_frame)
        button_frame.grid(row=17, column=0, columnspan=2, padx=10, pady=20)

        def handle_save():
            try:
                date_str = f"{year_var.get()}-{month_var.get()}-{day_var.get()}"
                datetime.datetime.strptime(date_str, '%Y-%m-%d')
                time_str = f"{time_var.get()}:00"
                duration = int(duration_var.get())
                appt_data = {
                    "date": date_str,
                    "time": time_str,
                    "duration": duration,
                    "status": status_var.get(),
                    "notes": notes_textbox.get("1.0", "end-1c").strip() or None
                }
                result = update_appointment(appointment_id, appt_data)
                if result["success"]:
                    messagebox.showinfo("Success", result["message"])
                    self.show_patient_details(patient_id)
                else:
                    error_label.configure(text=result["message"])
            except ValueError:
                error_label.configure(text="Invalid date format")

        save_button = ctk.CTkButton(
            button_frame,
            text="Update Appointment",
            command=handle_save
        )
        save_button.grid(row=0, column=0, padx=10, pady=10)

        cancel_button = ctk.CTkButton(
            button_frame,
            text="Cancel",
            fg_color="transparent",
            command=lambda: self.show_patient_details(patient_id)
        )
        cancel_button.grid(row=0, column=1, padx=10, pady=10)

    
    def load_appointments(self, filter_value):
        # Clear existing appointments
        for widget in self.appointments_container.winfo_children():
            widget.destroy()
        
        # Get appointments based on filter
        appointments = get_appointments(self.current_user["id"], filter_type=filter_value)
        
        if appointments:
            # Create a scrollable frame for the appointments
            scrollable_frame = ctk.CTkScrollableFrame(self.appointments_container, height=400)
            scrollable_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
            scrollable_frame.grid_columnconfigure(0, weight=1)
            
            # Table headers
            headers_frame = ctk.CTkFrame(scrollable_frame)
            headers_frame.grid(row=0, column=0, padx=5, pady=5, sticky="ew")
            headers_frame.grid_columnconfigure((0, 1, 2, 3, 4), weight=1)
            
            date_header = ctk.CTkLabel(headers_frame, text="Date", font=ctk.CTkFont(weight="bold"))
            date_header.grid(row=0, column=0, padx=5, pady=5, sticky="w")
            
            time_header = ctk.CTkLabel(headers_frame, text="Time", font=ctk.CTkFont(weight="bold"))
            time_header.grid(row=0, column=1, padx=5, pady=5, sticky="w")
            
            patient_header = ctk.CTkLabel(headers_frame, text="Patient", font=ctk.CTkFont(weight="bold"))
            patient_header.grid(row=0, column=2, padx=5, pady=5, sticky="w")
            
            duration_header = ctk.CTkLabel(headers_frame, text="Duration", font=ctk.CTkFont(weight="bold"))
            duration_header.grid(row=0, column=3, padx=5, pady=5, sticky="w")
            
            status_header = ctk.CTkLabel(headers_frame, text="Status", font=ctk.CTkFont(weight="bold"))
            status_header.grid(row=0, column=4, padx=5, pady=5, sticky="w")
            
            # List appointments
            for i, appt in enumerate(appointments):
                appt_frame = ctk.CTkFrame(scrollable_frame)
                appt_frame.grid(row=i+1, column=0, padx=5, pady=2, sticky="ew")
                appt_frame.grid_columnconfigure((0, 1, 2, 3, 4), weight=1)
                
                date_str = datetime.datetime.strptime(appt['date'], '%Y-%m-%d').strftime('%d/%m/%Y')
                date_str = parser.parse(note['date'], '%Y-%m-%d').strftime('%d %B %Y')
                date_label.grid(row=0, column=0, padx=5, pady=10, sticky="w")
                
                time_str = datetime.datetime.strptime(appt['time'], '%H:%M:%S').strftime('%I:%M %p')
                time_label = ctk.CTkLabel(appt_frame, text=time_str)
                time_label.grid(row=0, column=1, padx=5, pady=10, sticky="w")
                
                patient_label = ctk.CTkLabel(appt_frame, text=appt['patient_name'])
                patient_label.grid(row=0, column=2, padx=5, pady=10, sticky="w")
                
                duration_label = ctk.CTkLabel(appt_frame, text=f"{appt['duration']} min")
                duration_label.grid(row=0, column=3, padx=5, pady=10, sticky="w")
                
                status_label = ctk.CTkLabel(appt_frame, text=appt['status'].capitalize())
                status_label.grid(row=0, column=4, padx=5, pady=10, sticky="w")
                
                # View button
                view_button = ctk.CTkButton(
                    appt_frame,
                    text="View Patient",
                    width=100,
                    command=lambda p_id=appt['patient_id']: self.show_patient_details(p_id)
                )
                view_button.grid(row=0, column=5, padx=10, pady=10)
        else:
            no_appts_label = ctk.CTkLabel(
                self.appointments_container, 
                text="No appointments found with the selected filter."
            )
            no_appts_label.grid(row=0, column=0, padx=20, pady=20)

    def show_reports(self):
        self.clear_content()
        
        reports_frame = ctk.CTkFrame(self.content_frame)
        reports_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        reports_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = reports_frame
        
        title_label = ctk.CTkLabel(reports_frame, text="Reports", font=ctk.CTkFont(size=24, weight="bold"))
        title_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        patient_label = ctk.CTkLabel(reports_frame, text="Patient Report", font=ctk.CTkFont(weight="bold"))
        patient_label.grid(row=1, column=0, padx=20, pady=(10, 0), sticky="w")
        patient_init = ctk.CTkButton(reports_frame, text="Initial Report", command=self.generate_patient_initial_report)
        patient_init.grid(row=2, column=0, padx=40, pady=5, sticky="w")
        patient_progress = ctk.CTkButton(reports_frame, text="Progress Report", command=self.generate_patient_progress_report)
        patient_progress.grid(row=3, column=0, padx=40, pady=5, sticky="w")
        patient_discharge = ctk.CTkButton(reports_frame, text="Discharge Report", command=self.generate_patient_discharge_report)
        patient_discharge.grid(row=4, column=0, padx=40, pady=5, sticky="w")
        
        mva_label = ctk.CTkLabel(reports_frame, text="MVA Report", font=ctk.CTkFont(weight="bold"))
        mva_label.grid(row=5, column=0, padx=20, pady=(10, 0), sticky="w")
        mva_init = ctk.CTkButton(reports_frame, text="Initial Report", command=self.generate_mva_initial_report)
        mva_init.grid(row=6, column=0, padx=40, pady=5, sticky="w")
        mva_progress = ctk.CTkButton(reports_frame, text="Progress Report", command=self.generate_mva_progress_report)
        mva_progress.grid(row=7, column=0, padx=40, pady=5, sticky="w")
        mva_discharge = ctk.CTkButton(reports_frame, text="Discharge Report", command=self.generate_mva_discharge_report)
        mva_discharge.grid(row=8, column=0, padx=40, pady=5, sticky="w")

        motivational_label = ctk.CTkLabel(reports_frame, text="Motivational letter", font=ctk.CTkFont(weight="bold"))
        motivational_label.grid(row=9, column=0, padx=20, pady=(10, 0), sticky="w")
        motivational_button = ctk.CTkButton(reports_frame, text="Motivational Letter", command=self.generate_motivational_letter)
        motivational_button.grid(row=10, column=0, padx=40, pady=5, sticky="w")

    def generate_patient_initial_report(self):
        pass

    def generate_patient_progress_report(self):
        pass

    def generate_patient_discharge_report(self):
        pass

    def generate_mva_initial_report(self):
        pass

    def generate_mva_progress_report(self):
        pass

    def generate_mva_discharge_report(self):
        pass

    def generate_motivational_letter(self):
        pass

    def show_billing(self):
        self.clear_content()

        # Create billing frame
        billing_frame = ctk.CTkFrame(self.content_frame)
        billing_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        billing_frame.grid_columnconfigure(0, weight=1)
        self.current_frame = billing_frame

        # Title
        title_label = ctk.CTkLabel(
            billing_frame,
            text="Billing",
            font=ctk.CTkFont(size=24, weight="bold")
        )
        title_label.grid(row=0, column=0, padx=20, pady=(20, 10))


    def show_billing(self):
        self.clear_content()
        frame = ctk.CTkFrame(self.content_frame)
        frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        frame.grid_columnconfigure(0, weight=1)

        label = ctk.CTkLabel(frame, text="Billing module coming soon...", font=ctk.CTkFont(size=20))
        label.grid(row=0, column=0, padx=20, pady=20)


    def show_user_management(self):
        self.clear_content()

        frame = ctk.CTkFrame(self.content_frame)
        frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        frame.grid_columnconfigure(0, weight=1)
        self.current_frame = frame

        title = ctk.CTkLabel(frame, text="User Management", font=ctk.CTkFont(size=24, weight="bold"))
        title.grid(row=0, column=0, padx=20, pady=(20, 10), sticky="w")

        users = get_all_users()

        scrollable = ctk.CTkScrollableFrame(frame, height=400)
        scrollable.grid(row=1, column=0, padx=20, pady=20, sticky="nsew")
        scrollable.grid_columnconfigure((0,1,2,3), weight=1)

        header1 = ctk.CTkLabel(scrollable, text="Username", font=ctk.CTkFont(weight="bold"))
        header1.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        header2 = ctk.CTkLabel(scrollable, text="Name", font=ctk.CTkFont(weight="bold"))
        header2.grid(row=0, column=1, padx=5, pady=5, sticky="w")
        header3 = ctk.CTkLabel(scrollable, text="Role", font=ctk.CTkFont(weight="bold"))
        header3.grid(row=0, column=2, padx=5, pady=5, sticky="w")

        roles = ["manager", "administrator", "receptionist", "physiotherapist"]

        for i, user in enumerate(users):
            uname = ctk.CTkLabel(scrollable, text=user["username"])
            uname.grid(row=i+1, column=0, padx=5, pady=5, sticky="w")

            fullname = f"{user['first_name']} {user['last_name']}"
            name_lbl = ctk.CTkLabel(scrollable, text=fullname)
            name_lbl.grid(row=i+1, column=1, padx=5, pady=5, sticky="w")

            role_var = tk.StringVar(value=user["role"])
            role_menu = ctk.CTkOptionMenu(scrollable, variable=role_var, values=roles)
            role_menu.grid(row=i+1, column=2, padx=5, pady=5, sticky="w")

            def save_role(u_id=user["id"], var=role_var):
                result = update_user_role(u_id, var.get())
                if result["success"]:
                    messagebox.showinfo("Success", "Role updated")
                else:
                    messagebox.showerror("Error", result["message"])

            save_btn = ctk.CTkButton(scrollable, text="Save", width=80, command=save_role)
            save_btn.grid(row=i+1, column=3, padx=5, pady=5)

# Main function
if __name__ == "__main__":
    app = EiraNotesApp()
    
    # Set window icon if available
    # app.iconbitmap("icon.ico")  # Uncomment and provide path to icon if available
    
    app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\andil\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\andil\anaconda3\Lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 554, in _clicked
    self._command()
  File "C:\Users\andil\AppData\Local\Temp\ipykernel_18316\3976873652.py", line 1247, in show_patients
    dob = parser.parse(
          ^^^^^^^^^^^^^
  File "C:\Users\andil\anaconda3\Lib\site-packages\dateutil\parser\_parser.py", line 1366, in parse
    return parser(parserinfo).parse(timestr, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andil\anaconda3\Lib\site-packages\dateutil\parser\_parser.py", line 640, in parse
    res, skipped_tokens = self._parse(timestr, **kwargs)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andil\anaconda3\Lib\site-packages\dateutil\parser\_parser.py", line 